In [1]:
import requests
from bs4 import BeautifulSoup
import re
import csv
from csv import writer
import random
import time
import pandas as pd
import csv
import numpy as np
import math
import itertools
import concurrent.futures

In [2]:
#scrape from freeproxy.world
url = "https://www.freeproxy.world/"
proxylist =[]
#filter through all pages
for i in range(1,75):
    print(i)
    try:
        #filter to http proxy
        querystring = {"type":"http","anonymity":"","country":"","speed":"","port":"","page":f"{i}"}

        payload = ""
        headers = {"User-Agent": "insomnia/8.4.5"}

        #site request
        site = requests.request("GET", url, data=payload, headers=headers, params=querystring)
        soup = BeautifulSoup(site.content, 'html.parser')
        trs = soup.find_all('tr')
        for tr in trs:
            try:
                ip = tr.find('td',class_=re.compile('show-ip-div')).text.strip()
                port = tr.find('a').text.strip()
                proxylist.append(f'{ip}:{port}')
            except:
                pass
    except:
        pass
print(len(proxylist))




#scrape free proxies from proxyscrape.com
headers = {"User-Agent": "insomnia/8.4.5"}
#site request
url = 'https://proxyscrape.com/free-proxy-list'
site = requests.get(url,headers=headers)
soup = BeautifulSoup(site.content, 'html.parser')
#scrape http proxy download link
divs = soup.find_all('div', class_=re.compile('itemcard downloadcard'))
for div in divs:
    h2 = div.find('h2')
    try:
        if(h2.text.strip()=='HTTP Proxies'):
            a = div.find('a')
            download = a['download_url']
    except:
        pass

#get content of download link
try:
    site = requests.get(download,headers=headers)
    soup = BeautifulSoup(site.content, 'html.parser')
    #add proxies to testlist
    tempproxylist = soup.text.strip().split('\r')
    for proxy in tempproxylist:
        proxylist.append(proxy)

    print(len(proxylist))
except:
    pass    
#scrape proxies from free-proxy-list.net
url = 'https://free-proxy-list.net/'
#site + soup
site = requests.get(url, headers=headers)
soup = BeautifulSoup(site.content, 'html.parser')
#scrape table
try:
    table = soup.find('div', class_=re.compile('table-responsive'))
    trlabels = table.find_all('tr')
except:
    pass
#scrape proxies and add to test list
try:
    for i in trlabels:
            tdlabels = i.find_all('td')
            if(tdlabels[6].text.strip()=='yes'):
                proxylist.append(tdlabels[0].text.strip())
except:
    pass
print(len(proxylist))
#scrape proxies from hidemy.io
url = 'https://hidemy.io/en/proxy-list/'
#site request + soup
site = requests.get(url, headers=headers)
soup = BeautifulSoup(site.content, 'html.parser')
table = soup.find('div',class_=re.compile('table_block'))
#scrape table
trlabels = table.find_all('tr')
#scrape proxies and add to test list
for i in trlabels:
    tdlabels = i.find_all('td')
    try:

        if(tdlabels[4].text.strip()=='HTTP'):
            proxylist.append(tdlabels[0].text.strip())
    except:
        pass
print(len(proxylist))





#test proxies
workingProxies = []
def extract(proxy):
    works = False
    try:
        r = requests.get('https://www.whatismybrowser.com/detect/what-is-my-ip-address', proxies={'http':proxy, 'https':proxy}, timeout=2)
        soup = BeautifulSoup(r.content, 'html.parser')
        div = soup.find('div', class_='detected_result')
        ip = div.find('div').text.strip()
        print(f'{r}, {ip}')
        works = True
        workingProxies.append(proxy)
        with open('working_proxies.csv', 'w', encoding='UTF8', newline='') as workingProx:
            writer = csv.writer(workingProx)
            for k in workingProxies:
                writer.writerow([k])

    except:
        print(proxy)
    return works

#excecute test faster
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(extract, list(set(proxylist)))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
2132
2132
2196
23.227.38.230:80
14.56.98.15:3128
181.49.177.198:8080
24.152.40.49:8080
103.36.10.223:8080
122.255.58.227:80
162.223.89.84:80
143.110.232.177:80
104.27.122.6:80
20.44.190.150:3129
45.11.95.166:6009
223.25.100.237:8080
51.210.127.15:80
161.132.48.32:8080
41.207.187.178:80
213.97.158.42:3128
50.200.12.80:80
103.165.128.171:8080
103.42.57.13:3128
103.169.255.181:8080
41.159.154.43:3128
50.207.199.87:80
185.169.183.200:8080
103.115.29.65:8080
185.74.6.249:8080
216.176.187.99:39837
51.15.242.202:8888
103.189.249.163:1111
89.116.34.113:80
113.160.247.27:19132
124.83.35.42:8181
45.178.133.75:999
104.25.194.175:80
83.98.243.181:80
103.173.128.50:8080
46.105.42.230:3128
185.162.230.136
45.176.95.38:999
173.245.49.27:80
167.250.222.233:999
36.95.142.35:8080
185.225.232.19

In [3]:
proxylist =[]
with open('working_proxies.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        proxylist.append(row[0])
print(f'Proxies: {len(proxylist)}')

Proxies: 40


In [4]:
#ensure site is accessible
def getProxyUserAgent():
    for i in proxylist:
        #test to see if website is accessible
        userAgents = ['Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/116.0','Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/115.0','Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36','Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36','Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36','Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0','Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/116.0','Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.188','insomnia/8.4.5','Mozilla/5.0 (Linux; Android 13; Pixel 6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Mobile Safari/537.36','Mozilla/5.0 (Linux; Android 13; Pixel 6a) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Mobile Safari/537.36','Mozilla/5.0 (Linux; Android 13; Pixel 6 Pro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Mobile Safari/537.36','Mozilla/5.0 (Linux; Android 13; Pixel 7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Mobile Safari/537.36','Mozilla/5.0 (Linux; Android 13; Pixel 7 Pro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Mobile Safari/537.36','Mozilla/5.0 (Linux; Android 12; moto g pure) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Mobile Safari/537.36','Mozilla/5.0 (Linux; Android 12; moto g stylus 5G) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Mobile Safari/537.36v','Mozilla/5.0 (Linux; Android 13; SM-G998U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Mobile Safari/537.36','Mozilla/5.0 (iPhone; CPU iPhone OS 12_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) FxiOS/13.2b11866 Mobile/16A366 Safari/605.1.15','Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/11.0 Mobile/15A372 Safari/604.1']
        userAgent = userAgents[random.randint(0,len(userAgents)-1)]
        try:
            payload = ''
            headers = {
                "cookie": "_tapology_mma_session=zo%252F9cU1na2qfRAvGf4E%252FBdxyMqNEgZbmqUYGnDxTYw%252BkgkuquO4qPimSMq%252FNc4fAxpLoIGwkl%252Fvw%252FhO04rqrL1PuS7516fTWktFyWhkz6YUy7MvWUVyjNQ7R26QYA8TeQruG5w%252B6RAj71bMME5MxoLjpSOs%252FyinaA6qsprmBZ2LnagrpzZ7bxaPvk7o%252FohTgZgxpo0FGWGaDdHERD%252B3Bt3C3ucykGOC7WB65EM8xB6C8gMNzGsvEu8FbvGnbMaoqzGzx%252FjRprzFrLN4mE5vdrJ1fsjoDV9cn5NEE2zI%253D--IkSvjCRiM5qMGaKi--o55iUwoIEhI6T8jM4UW6tA%253D%253D",
                "User-Agent": f'{userAgent}'
            }

            url = "https://www.tapology.com"
        
            #site request
            response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{i}"})
            soup = BeautifulSoup(response.content, 'html.parser')
            boutInfo = soup.find('div', class_=re.compile('right'))
            labels = boutInfo.find_all('li')
            proxyheader = [i,userAgent]
            return proxyheader
        except:
            pass
        #check to see if user agent was the issue
        try:
            userAgents.remove(userAgent)
            userAgent = userAgents[random.randint(0,len(userAgents)-1)]
            payload = ''
            headers = {'User-Agent':f'{userAgent}',
            "cookie": "_tapology_mma_session=TCLB17ieOPnBLmCBTuxpX8s3uBODZMN3jL3jBbFhwPoywfbzG7gyvp%252BAzbOk4gOZ%252FOCykOUwcpEoJBwoj2rJyiMxdHWSaiLFkBYjfuUDpZ2VY6ECFn6rpTPmUBY1Zr2anIqiklY6fz9yQlBkPAhcx%252BWSzVgsc%252B%252F8UCqkb6WnM6xr8GUikb8U2UkMVYZ3Nj1dIA0vbXpDhKykqgW%252BCnlyglp8rtdlQ37m0SaYWjLDthG7Tik3idUvGlSXFAU55zAnxz6UNncMNhTbo5ltINfso54j60i7hOq0utNOz9w%253D--ChZexYwNpevIJ%252BV8--HhKStLELFfNWYOTZIAKM6Q%253D%253D"}
            url = "https://www.tapology.com/fightcenter/bouts/2974-ufc-64-clay-the-carpenter-guida-vs-justin-pretty-boy-james"
        
            #site request
            response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{i}"})
            soup = BeautifulSoup(response.content, 'html.parser')
            boutInfo = soup.find('div', class_=re.compile('right'))
            labels = boutInfo.find_all('li')
            proxyheader = [i,userAgent]
            return proxyheader
        #if user agent is not new issue, wait for next IP
        except:
            pass
        try:
            print("Waiting for new IP...")
            ipurl = "https://ipecho.net/plain"
            ipHeader = {"User-Agent": "insomnia/8.4.5"}
            ipr = requests.request("GET", url=ipurl, headers=ipHeader)
            soup = BeautifulSoup(ipr.content, 'html.parser')
            currentIp = soup.text.strip()
            newIP = soup.text.strip()
            while(currentIp == newIP):
                ipurl = "https://ipecho.net/plain"
                ipHeader = {"User-Agent": "insomnia/8.4.5"}
                ipr = requests.request("GET", url=ipurl, headers=ipHeader)
                soup = BeautifulSoup(ipr.content, 'html.parser')
                newIP = soup.text.strip()
                time.sleep(15)
            userAgents.remove(userAgent)
            userAgent = userAgents[random.randint(0,len(userAgents)-1)]
            payload = ''
            headers = {'User-Agent':f'{userAgent}',
            "cookie": "_tapology_mma_session=TCLB17ieOPnBLmCBTuxpX8s3uBODZMN3jL3jBbFhwPoywfbzG7gyvp%252BAzbOk4gOZ%252FOCykOUwcpEoJBwoj2rJyiMxdHWSaiLFkBYjfuUDpZ2VY6ECFn6rpTPmUBY1Zr2anIqiklY6fz9yQlBkPAhcx%252BWSzVgsc%252B%252F8UCqkb6WnM6xr8GUikb8U2UkMVYZ3Nj1dIA0vbXpDhKykqgW%252BCnlyglp8rtdlQ37m0SaYWjLDthG7Tik3idUvGlSXFAU55zAnxz6UNncMNhTbo5ltINfso54j60i7hOq0utNOz9w%253D--ChZexYwNpevIJ%252BV8--HhKStLELFfNWYOTZIAKM6Q%253D%253D"}
            url = "https://www.tapology.com/fightcenter/bouts/2974-ufc-64-clay-the-carpenter-guida-vs-justin-pretty-boy-james"
        
            #site request
            response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{i}"})
            soup = BeautifulSoup(response.content, 'html.parser')
            boutInfo = soup.find('div', class_=re.compile('right'))
            labels = boutInfo.find_all('li')
            proxyheader = [i,userAgent]
            return proxyheader
        except:
            print("Maintenence required...")
            input("Press enter to continue")
            userAgents.remove(userAgent)
            userAgent = userAgents[random.randint(0,len(userAgents)-1)]
            payload = ''
            headers = {'User-Agent':f'{userAgent}',
            "cookie": "_tapology_mma_session=TCLB17ieOPnBLmCBTuxpX8s3uBODZMN3jL3jBbFhwPoywfbzG7gyvp%252BAzbOk4gOZ%252FOCykOUwcpEoJBwoj2rJyiMxdHWSaiLFkBYjfuUDpZ2VY6ECFn6rpTPmUBY1Zr2anIqiklY6fz9yQlBkPAhcx%252BWSzVgsc%252B%252F8UCqkb6WnM6xr8GUikb8U2UkMVYZ3Nj1dIA0vbXpDhKykqgW%252BCnlyglp8rtdlQ37m0SaYWjLDthG7Tik3idUvGlSXFAU55zAnxz6UNncMNhTbo5ltINfso54j60i7hOq0utNOz9w%253D--ChZexYwNpevIJ%252BV8--HhKStLELFfNWYOTZIAKM6Q%253D%253D"}
            url = "https://www.tapology.com/fightcenter/bouts/2974-ufc-64-clay-the-carpenter-guida-vs-justin-pretty-boy-james"
        
            #site request
            response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{i}"})
            soup = BeautifulSoup(response.content, 'html.parser')
            boutInfo = soup.find('div', class_=re.compile('right'))
            labels = boutInfo.find_all('li')
            proxyheader = [i,userAgent]
            return proxyheader

In [5]:
#site request
proxyheader = getProxyUserAgent()
proxy = proxyheader[0]
userAgent = proxyheader[1]
querystring = {"group":"ufc","page":1,"region":"","schedule":"results","sport":"all"}
url = 'https://www.tapology.com/fightcenter'

payload = ""
headers = {
    "cookie": "_tapology_mma_session=%252BnGrxOO8u60FBkwjnPf5U9cMUlW%252B%252F76dZtFNqnrNzBiOvQybvXmEnNM%252Fu1%252BEvOx0w4zOYLO6aIlNCfl8UnsrtSYiMl2eRJHAyiBcnd2iP0A0MCwFxGErsRcK9jbT%252BixWWetj2aX%252FvsQSBYea%252Fe73CRDIdSn95lPxaMgzhrkIGIY2KzurUSeLm0hoWxHQyq01nb7UJfYbF53mL1vhZO1yAYpprixBeuhXy70HLYlQemANkpVvl7tT0Z5DTe68LgVyn8qXKLn1hOvclfkBIfaVwBd1HyV5eIRqOMicbIQ%253D--%252BzoeexZ3ARMoGYGy--kqTFUNtOGR74GqxibycT1g%253D%253D",
    "User-Agent": f"{userAgent}"
}

response = requests.request("GET", url, data=payload, headers=headers, params=querystring, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})
print(response)


<Response [200]>


In [6]:
#get part for url of event, range defines number of events to scrape
parts0 = []
for i in range(0, 1):
    soup = BeautifulSoup(response.content, 'html.parser')
    az = soup.find_all('a', class_=re.compile('border-b border-tap_3 border-dotted hover:border-solid'))
    parts0.append(az[i]['href'])

parts = []
for part in parts0:
    #site request
    proxyheader = getProxyUserAgent()
    proxy = proxyheader[0]
    userAgent = proxyheader[1]
    headers = {
        "cookie": "_tapology_mma_session=SalX0GSlJmX83Xr61Esyd2IpHEnEaKZ8EYxG%252FzqzxFw61SFd5cUeTP0bI6faVXCwYrNZI%252FCI%252F8k3ulCobIDgFsfjQS0mH2Cmyrjw8uRPRid1zHpuRUdB38T9zitk9HAt06s%252BJfGPJHcaekBUI5HjpDKqJqiMNg7codsNhLvZcnHeW1FlGhcz%252BGEtmlLZDuRBcl2UPylh%252B4x97JplnC7%252FxEOXfQg51XgXLvIBL4dbuO90Cwblj7LJe3XnNpqLWcefA4r6d9gVHHKBiRN10S1K9ntnmMjzz%252FLTgDxPUBI%253D--cIDn0iKYjggibUZ3--CAvjxd4oiPcvmYyMijxPGA%253D%253D",
        "User-Agent": f'{userAgent}'
    }

    url = f'https://www.tapology.com{part}'
    print(url)
    site = requests.get(url, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})
    #get url parts from all fights on the card
    soup = BeautifulSoup(site.content, 'html.parser')
    spans = soup.find_all('span', class_=re.compile('text-xs11 md:text-xs10 uppercase font-bold'))
    for span in spans:
        href = span.find('a')
        part = href['href']
        parts.append(part)
print(parts) 



https://www.tapology.com/fightcenter/events/108288-ufc-fight-night
['/fightcenter/bouts/835883-ufc-fight-night-tai-bam-bam-tuivasa-vs-marcin-tybur-tybura', '/fightcenter/bouts/835212-ufc-fight-night-bryan-pooh-bear-battle-vs-ange-the-last-ninja-loosa', '/fightcenter/bouts/836823-ufc-fight-night-kennedy-african-savage-nzechukwu-vs-ovince-osp-st-preux', '/fightcenter/bouts/834662-ufc-fight-night-christian-ceerod-rodriguez-vs-isaac-the-midwest-choppa-dulgarian', '/fightcenter/bouts/838614-ufc-fight-night-pannie-banzai-kianzad-vs-macy-chiasson-ii', '/fightcenter/bouts/838612-ufc-fight-night-gerald-gm3-meerschaert-vs-bryan-bam-bam-barberena', '/fightcenter/bouts/838613-ufc-fight-night-mike-beast-boy-davis-vs-natan-lethal-levy', '/fightcenter/bouts/838615-ufc-fight-night-josiane-josi-nunes-vs-chelsea-chandler', '/fightcenter/bouts/826263-ufc-fight-night-ode-the-jamaican-sensation-osbourne-vs-jafel-pastor-filho', '/fightcenter/bouts/838618-ufc-fight-night-josh-kuya-culibao-vs-danny-el-puma-si

In [7]:
fightStats = []

for i in parts:
    proxyheader = getProxyUserAgent()
    proxy = proxyheader[0]
    userAgent = proxyheader[1]
    payload = ""
    headers = {
        "cookie": "_tapology_mma_session=6FGp%252FJUSTWYMfoxo8TfW%252BIXzLpUq7U9PMAJo5rHJA0IW5nmUvBfyvSfM1xK04kt35b9X7qEKQCxCoWu2ufxYHMbwDH88yla0%252FpzMP71n6pbfW%252FroMtWAh2n5sk9oxYFnmpfxohRaQMysmv%252B9f5fj0Omemblq8KM9NEDFiR5UPFyFXXYiM0Ee%252FWLYZ5JqObzpWnulDsrgvVtdtWFthH9vY6xz9HAvSb4KOm1HA6TvXXxYOO2Vuk6MeJwKYdwj3yqz8dV%252FHRgPknI5PsGEx3z3mxBNOJaFkRBT6iB%252B5Zo%253D--pNnhVuV4ORLaBVOE--yUSbtIv6C91epo3hOY0i5w%253D%253D",
        "User-Agent": f'{userAgent}'
    }


    url = f"https://www.tapology.com{i}"
        
    #site request
    response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})
    soup = BeautifulSoup(response.content, 'html.parser')

    
    try:
        #scrape event details
        lilabels = soup.find_all('li', class_=re.compile('even:bg-tap_f2 leading-normal py-1.5 md:py-2 px-1 md:text-xs'))
    except:
        proxyheader = getProxyUserAgent()
        proxy = proxyheader[0]
        userAgent = proxyheader[1]
        payload = ""
        headers = {
            "cookie": "_tapology_mma_session=6FGp%252FJUSTWYMfoxo8TfW%252BIXzLpUq7U9PMAJo5rHJA0IW5nmUvBfyvSfM1xK04kt35b9X7qEKQCxCoWu2ufxYHMbwDH88yla0%252FpzMP71n6pbfW%252FroMtWAh2n5sk9oxYFnmpfxohRaQMysmv%252B9f5fj0Omemblq8KM9NEDFiR5UPFyFXXYiM0Ee%252FWLYZ5JqObzpWnulDsrgvVtdtWFthH9vY6xz9HAvSb4KOm1HA6TvXXxYOO2Vuk6MeJwKYdwj3yqz8dV%252FHRgPknI5PsGEx3z3mxBNOJaFkRBT6iB%252B5Zo%253D--pNnhVuV4ORLaBVOE--yUSbtIv6C91epo3hOY0i5w%253D%253D",
            "User-Agent": f'{userAgent}'
        }
        response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})
        soup = BeautifulSoup(response.content, 'html.parser')
        #scrape event details
        lilabels = soup.find_all('li', class_=re.compile('even:bg-tap_f2 leading-normal py-1.5 md:py-2 px-1 md:text-xs'))

        
    print(url)
    #initialize attributes
    event = None
    date = None
    venue = None
    title_fight = 'no'
    billing = None
    winner = None
    loser = None
    winner_wins = None
    loser_wins = None
    winner_losses = None
    loser_losses = None
    winner_draws = None
    loser_draws = None
    winner_age = None
    loser_age = None
    belt_status = None
    winner_nationality = None
    loser_nationality = None
    winner_fan = None
    loser_fan = None
    fight_name = None


    for num in range(0, len(lilabels)+1):
        #scrape + clean event name
        try:
            if(lilabels[num].find('span', class_=re.compile('font-bold text-neutral-900')).text.strip() == 'Event:'):
                event = lilabels[num].find('a', class_=re.compile('link-primary-red')).text.strip()
        except:
            pass
        #scrape + clean date
        try:
            if(lilabels[num].find('span', class_=re.compile('font-bold text-neutral-900')).text.strip() == 'Date:'):
                nCleanDate = lilabels[num].find('span', class_=re.compile('text-neutral-700')).text.strip()
                dateList = nCleanDate.split(' ')
                date = dateList[1]
        except:
            pass
        #scrape + clean venue
        try:
            if(lilabels[num].find('span', class_=re.compile('font-bold text-neutral-900')).text.strip() == 'Venue:'):
                venue = lilabels[num].find('span', class_=re.compile('text-neutral-700')).text.strip()
        except:
            pass
        #scrape whether it is a title fight
        try:
            if(lilabels[num].find('span', class_=re.compile('font-bold text-neutral-90')).text.strip() == 'Title on Line:'):
                title_fight = 'yes'
        except:
            pass
        #scrape + clean billing
        try:
            if(lilabels[num].find('span', class_=re.compile('font-bold text-neutral-900')).text.strip() == 'Bout Billing:'):
                billing = lilabels[num].find('span', class_=re.compile('text-neutral-700')).text.strip()
        except:
            pass
    
    #scrape +clean winner
    try:
        winnerCheck = soup.find('span', class_=re.compile('text-lg leading-tight text-tap_3 font-bold')).text.strip()
        if('defeats' in winnerCheck):
            fighters = soup.find_all('a', class_=re.compile('link-primary-red hidden md:inline'))
            winner = fighters[0].text.strip()
            loser = fighters[1].text.strip()
    except:
        pass

    #scrape + parse table
    table = soup.find('table', class_=re.compile('md:mt-5 w-full text-center'))
    tdLabels = table.find_all('td')

    #scrape + clean winner record
    try:
        winnerRecordTempList = tdLabels[0].text.strip().split('\n')
        winnerRecord = winnerRecordTempList[0]
        winnerRecordList = winnerRecord.split('-')
        winner_wins = winnerRecordList[0]
        winner_losses = winnerRecordList[1]
        winner_draws = winnerRecordList[2]
    except:
        pass
    
    #scrape + clean loser record
    try:
        loserRecordTempList = tdLabels[5].text.strip().split('\n')
        loserRecord = loserRecordTempList[0]
        loserRecordList = loserRecord.split('-')
        loser_wins = loserRecordList[0]
        loser_losses = loserRecordList[1]
        loser_draws = loserRecordList[2]
    except:
        pass
    
    #scrape and parse table
    trLabels = table.find_all('tr')
    for tr in trLabels:
        #scrape + clean winner and loser age
        try:
            if('Age at Fight' in tr.text.strip()):
                td = tr.find_all('td')
                winner_ageTemp = td[0].text.strip().split(' ')
                winner_age = winner_ageTemp[0]
                loser_ageTemp = td[5].text.strip().split(' ')
                loser_age = loser_ageTemp[0]
        except:
            pass
        #scrape + clean nationality
        try:
            if('Nation' in tr.text.strip()):
                td = tr.find_all('td')
                winner_nationTemp = td[0].text.strip().split('\n')
                winner_nationality = winner_nationTemp[0]
                loser_nationTemp = td[4].text.strip().split('\n')
                loser_nationality = loser_nationTemp[0]
        except:
            pass
            

    #scrape + clean fight name
    try:
        fight_name = soup.find('h2', class_=re.compile('text-2xl md:text-2xl text-center font-bold text-tap_3')).text.strip()
    except:
        fight_name = soup.find('h2', class_=re.compile('text-xl md:text-2xl text-center font-bold text-tap_3')).text.strip()

    print(f'Scraping {fight_name}...')
    fightStats.append([fight_name,winner, loser, event,date,venue,title_fight,billing,winner_wins ,loser_wins ,winner_losses ,loser_losses ,winner_draws,loser_draws,winner_age ,loser_age ,belt_status ,winner_nationality ,loser_nationality ,winner_fan ,loser_fan])

     
         

https://www.tapology.com/fightcenter/bouts/835883-ufc-fight-night-tai-bam-bam-tuivasa-vs-marcin-tybur-tybura
Scraping Tai Tuivasa vs. Marcin Tybura...
https://www.tapology.com/fightcenter/bouts/835212-ufc-fight-night-bryan-pooh-bear-battle-vs-ange-the-last-ninja-loosa
Scraping Bryan Battle vs. Ange Loosa...
https://www.tapology.com/fightcenter/bouts/836823-ufc-fight-night-kennedy-african-savage-nzechukwu-vs-ovince-osp-st-preux
Scraping Kennedy Nzechukwu vs. Ovince St. Preux...
https://www.tapology.com/fightcenter/bouts/834662-ufc-fight-night-christian-ceerod-rodriguez-vs-isaac-the-midwest-choppa-dulgarian
Scraping Christian Rodriguez vs. Isaac Dulgarian...
https://www.tapology.com/fightcenter/bouts/838614-ufc-fight-night-pannie-banzai-kianzad-vs-macy-chiasson-ii
Scraping Pannie Kianzad vs. Macy Chiasson II...
https://www.tapology.com/fightcenter/bouts/838612-ufc-fight-night-gerald-gm3-meerschaert-vs-bryan-bam-bam-barberena
Scraping Gerald Meerschaert vs. Bryan Barberena...
https://www.

In [8]:
head = ['fight','winner', 'loser', 'event','date','venue','title_fight','billing','winner_wins' ,'loser_wins' ,'winner_losses' ,'loser_losses' ,'winner_draws','loser_draws','winner_age' ,'loser_age' ,'belt_status' ,'winner_nationality' ,'loser_nationality' ,'winner_fan ','loser_fan']


with open('updateTapology.csv', 'w', encoding='UTF8', newline='') as updateTapology:
    writer = csv.writer(updateTapology)
    writer.writerow(head)
    writer.writerows(fightStats)

In [9]:
#define url and header
url = 'http://ufcstats.com/statistics/events/completed?page=all'
headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}

#site request
site = requests.get(url, headers=headers)
print(site)

<Response [200]>


In [10]:
#soup and get hrefs
soup = BeautifulSoup(site.content, 'html.parser')
hrefs = soup.find_all('a', class_=re.compile('b-link b-link_style_black'))

#get link for most recent event
link = hrefs[0]['href']

#get part for url of event, range defines number of events to scrape
links = []
for i in range(0, 1):
    link = hrefs[i]['href']
    links.append(link)

print(links)


['http://ufcstats.com/event-details/c398235fcaf8d71d']


In [11]:
stat_links = []
for link in links:
    #site request
    site = requests.get(link, headers=headers)
    soup = BeautifulSoup(site.content, 'html.parser')

    #scrape stat links
    stat_linksTemp = soup.find_all('a', class_=['b-flag b-flag_style_green', 'b-flag b-flag_style_bordered'])
    for statLink in stat_linksTemp:
        stat_links.append(statLink)

#clean stat links
statLinks = []
for link in stat_links:
    href = link['href']
    if href:
        statLinks.append(href)

In [12]:
#remove dups
statLinks = list(dict.fromkeys(statLinks)) 
print(f'Fights found: {len(statLinks)}')

Fights found: 13


In [13]:
#scrape stats
fightStats = []
count = 0
for i in statLinks:

    #initialize attributes
    redCorner = None
    blueCorner = None
    winner = None
    event = None
    referee = None
    method_of_victory = None
    red_Knockdowns = None
    blue_Knockdowns = None
    red_sig_str = None
    blue_sig_str = None
    red_sig_str_percentage = None
    blue_sig_str_percentage = None
    red_total_strikes = None
    blue_total_strikes = None
    red_takedowns = None
    blue_takedowns = None
    red_takedown_percentage = None
    blue_takedown_percentage = None
    red_subs_attempted = None
    blue_subs_attempted = None
    round = None
    time = None
    redCorner_height = None
    blueCorner_height = None
    redCorner_reach = None
    blueCorner_reach = None
    redCorner_stance = None
    blueCorner_stance = None

    #site request
    try:
        site = requests.get(i, headers=headers)
        soup = BeautifulSoup(site.content, 'html.parser')
    except:
        pass
    
    #scrape corners
    try:
        red_corner = soup.find_all('i', class_=re.compile('b-fight-details__charts-name b-fight-details__charts-name_pos_left js-chart-name'))
    except:
        pass
    try:
        blue_corner = soup.find_all('i', class_=re.compile('b-fight-details__charts-name b-fight-details__charts-name_pos_right js-chart-name'))
    except:
        pass

    #clean corners
    try:
        redCorner = red_corner[0].text.strip()
    except:
        pass
    try:
        blueCorner = blue_corner[0].text.strip()
    except:
        pass

    #scrape + clean winner
    try:
        divs = soup.find_all('div', class_=re.compile('b-fight-details__person'))
    except:
        pass
    try:
        if(divs[1].find('i', class_='b-fight-details__person-status b-fight-details__person-status_style_green') != None):
            winner = divs[1].find('a', class_=re.compile('b-link b-fight-details__person-link')).text.strip()
        if(divs[3].find('i', class_='b-fight-details__person-status b-fight-details__person-status_style_green') != None):
            winner = divs[3].find('a', class_=re.compile('b-link b-fight-details__person-link')).text.strip()
    except:
        pass
    
    #scrape +clean event
    try:
        event = soup.find('a', class_=re.compile('b-link')).text.strip()
    except:
        pass

    #scrape + clean method of vitory
    try:
        method_of_victory = soup.find('i', style='font-style: normal').text.strip()
    except:
        pass

    #scrape + clean referee
    try:
        referee = soup.find('span').text.strip()
    except:
        pass
    
    #scrape + clean round and time
    iz = soup.find_all('i', class_=re.compile('b-fight-details__text-item'))
    for j in range(len(iz)):
        details = iz[j].find('i', class_=re.compile('b-fight-details__label'))
        try:
            test = details.text.strip()
            if "Round:" in test:
                round = iz[j].text.strip().replace(' ', '').replace('R', '').replace('o', '').replace('u', '').replace('n', '').replace('d', '').replace(':', '').replace("\n", '')
            if "Time:" in test:
                unFormattedTime = iz[j].text.strip().replace(' ', '').replace('T', '').replace('i', '').replace('m', '').replace('e', '').replace(':', '').replace("\n", '')
                time = unFormattedTime[0] + ":" + unFormattedTime[1:]
        except:
            pass

    #scrape red blue stats
    try:
        ps = soup.find_all('p', class_='b-fight-details__table-text')
    except:
        pass
    
    try:
        #scrape + clean knockdowns
        red_Knockdowns = ps[2].text.strip()
        blue_Knockdowns = ps[3].text.strip()

        #scrape + clean sig strikes
        red_sig_str = ps[4].text.strip()
        blue_sig_str = ps[5].text.strip()

        #scrape + clean sig strike percentage
        red_sig_str_percentage = ps[6].text.strip()
        blue_sig_str_percentage = ps[7].text.strip()

        #scrape + clean total strikes
        red_total_strikes = ps[8].text.strip()
        blue_total_strikes = ps[9].text.strip()

        #scrape + clean takedowns
        red_takedowns = ps[10].text.strip()
        blue_takedowns = ps[11].text.strip()

        #scrape + clean takedown percentage
        red_takedown_percentage = ps[12].text.strip()
        blue_takedown_percentage = ps[13].text.strip()

        #scrape + clean
        red_subs_attempted = ps[14].text.strip()
        blue_subs_attempted = ps[15].text.strip()
    except:
        pass
    
    try:
        #scrape redCorner/blueCorner links 
        indstatLinks = soup.find_all('a', class_=re.compile('b-link b-link_style_black'))
        redCornerLink = indstatLinks[0]
        blueCornerLink = indstatLinks[1]
        
        #clean for href
        redCornerLink = redCornerLink['href']
        blueCornerLink = blueCornerLink['href']

        response = requests.get(redCornerLink, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        #clean soup for rest of stats
        i_tags = soup.find_all('i')
        for itags in i_tags:
            itags.decompose()
        tempRest = soup.find_all('li', class_=re.compile('b-list__box-list-item b-list__box-list-item_type_block'))
        redCorner_height = tempRest[0].text.strip()
        redCorner_reach = tempRest[2].text.strip()
        redCorner_stance = tempRest[3].text.strip()

        #height, reach, stance blue
        response = requests.get(blueCornerLink, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        #clean soup for rest of stats
        i_tags = soup.find_all('i')
        for itags in i_tags:
            itags.decompose()
        tempRest = soup.find_all('li', class_=re.compile('b-list__box-list-item b-list__box-list-item_type_block'))
        blueCorner_height = tempRest[0].text.strip()
        blueCorner_reach = tempRest[2].text.strip()
        blueCorner_stance = tempRest[3].text.strip()
    except:
        pass
    fightStats.append([redCorner, blueCorner, winner, event, referee, method_of_victory, red_Knockdowns, blue_Knockdowns, red_sig_str, blue_sig_str, red_sig_str_percentage, blue_sig_str_percentage, red_total_strikes, blue_total_strikes, red_takedowns, blue_takedowns, red_takedown_percentage, blue_takedown_percentage, red_subs_attempted, blue_subs_attempted, round, time, redCorner_height, blueCorner_height, redCorner_reach, blueCorner_reach, redCorner_stance, blueCorner_stance])


    print(f'Sraping {redCorner} vs {blueCorner}...{f"{count/len(statLinks):.0%}"}')
    print(i)
    count+=1  


Sraping Tai Tuivasa vs Marcin Tybura...0%
http://ufcstats.com/fight-details/8bc0c220f7dcefa7
Sraping Bryan Battle vs Ange Loosa...8%
http://ufcstats.com/fight-details/a7fd9da657bbce8e
Sraping Ovince Saint Preux vs Kennedy Nzechukwu...15%
http://ufcstats.com/fight-details/35f9beb94b4ef9e2
Sraping Christian Rodriguez vs Isaac Dulgarian...23%
http://ufcstats.com/fight-details/acc7392d48cc09a0
Sraping Pannie Kianzad vs Macy Chiasson...31%
http://ufcstats.com/fight-details/2f414494f2b6f426
Sraping Gerald Meerschaert vs Bryan Barberena...38%
http://ufcstats.com/fight-details/569e0ba18701c305
Sraping Natan Levy vs Mike Davis...46%
http://ufcstats.com/fight-details/4f20f3ada1489823
Sraping Josiane Nunes vs Chelsea Chandler...54%
http://ufcstats.com/fight-details/711ec0545552455b
Sraping Jafel Filho vs Ode Osbourne...62%
http://ufcstats.com/fight-details/451df53c13185584
Sraping Josh Culibao vs Danny Silva...69%
http://ufcstats.com/fight-details/c26f27667c230391
Sraping Jaqueline Amorim vs Cory

In [14]:
#create csv file
head = ['redCorner', 'blueCorner', 'winner', 'event', 'referee', 'method_of_victory', 'red_Knockdowns', 'blue_Knockdowns', 'red_sig_str', 'blue_sig_str', 'red_sig_str_percentage', 'blue_sig_str_percentage', 'red_total_strikes', 'blue_total_strikes', 'red_takedowns', 'blue_takedowns', 'red_takedown_percentage', 'blue_takedown_percentage', 'red_subs_attempted', 'blue_subs_attempted', 'round', 'time', 'redCorner_height', 'blueCorner_height', 'redCorner_reach', 'blueCorner_reach', 'redCorner_stance', 'blueCorner_stance']

with open('updateUfc_history_fight_statistics.csv', 'w', encoding='UTF8', newline='') as updateHistory:
    writer = csv.writer(updateHistory)
    writer.writerow(head)
    writer.writerows(fightStats)

In [2]:
#create dataframes
df = pd.read_csv('updateTapology.csv')
df2 = pd.read_csv('updateUfc_history_fight_statistics.csv')
df.head()

,fight,winner,loser,event,date,venue,title_fight,billing,winner_wins,loser_wins,...,loser_losses,winner_draws,loser_draws,winner_age,loser_age,belt_status,winner_nationality,loser_nationality,winner_fan,loser_fan
0,Tai Tuivasa vs. Marcin Tybura,Marcin Tybura,Tai Tuivasa,UFC Fight Night: Tuivasa vs. Tybura,03.16.2024,UFC Apex,no,Main Event,24,15,...,6,0,0,38,31,NaN,Poland,Australia,NaN,NaN
1,Bryan Battle vs. Ange Loosa,NaN,NaN,UFC Fight Night: Tuivasa vs. Tybura,03.16.2024,UFC Apex,no,Co-Main Event,10,10,...,3,0,0,29,29,NaN,United States,Switzerland,NaN,NaN
2,Kennedy Nzechukwu vs. Ovince St. Preux,Ovince St. Preux,Kennedy Nzechukwu,UFC Fight Night: Tuivasa vs. Tybura,03.16.2024,UFC Apex,no,Main Card,26,12,...,4,0,0,40,31,NaN,Haiti,Nigeria,NaN,NaN
3,Christian Rodriguez vs. Isaac Dulgarian,Christian Rodriguez,Isaac Dulgarian,UFC Fight Night: Tuivasa vs. Tybura,03.16.2024,UFC Apex,no,Main Card,10,6,...,0,0,0,26,27,NaN,United States,United States,NaN,NaN
4,Pannie Kianzad vs. Macy Chiasson II,Macy Chiasson,Pannie Kianzad,UFC Fight Night: Tuivasa vs. Tybura,03.16.2024,UFC Apex,no,Main Card,8,16,...,7,0,0,32,32,NaN,United States,Sweden,NaN,NaN


In [3]:
#match event names
for (index, row), (index2, row2) in zip(df.iterrows(), df2.iterrows()):
        if(row['event'] != row2['event']):
            df.at[index, 'event'] = row2['event']


In [4]:
df.head()

,fight,winner,loser,event,date,venue,title_fight,billing,winner_wins,loser_wins,...,loser_losses,winner_draws,loser_draws,winner_age,loser_age,belt_status,winner_nationality,loser_nationality,winner_fan,loser_fan
0,Tai Tuivasa vs. Marcin Tybura,Marcin Tybura,Tai Tuivasa,UFC Fight Night: Tuivasa vs. Tybura,03.16.2024,UFC Apex,no,Main Event,24,15,...,6,0,0,38,31,NaN,Poland,Australia,NaN,NaN
1,Bryan Battle vs. Ange Loosa,NaN,NaN,UFC Fight Night: Tuivasa vs. Tybura,03.16.2024,UFC Apex,no,Co-Main Event,10,10,...,3,0,0,29,29,NaN,United States,Switzerland,NaN,NaN
2,Kennedy Nzechukwu vs. Ovince St. Preux,Ovince St. Preux,Kennedy Nzechukwu,UFC Fight Night: Tuivasa vs. Tybura,03.16.2024,UFC Apex,no,Main Card,26,12,...,4,0,0,40,31,NaN,Haiti,Nigeria,NaN,NaN
3,Christian Rodriguez vs. Isaac Dulgarian,Christian Rodriguez,Isaac Dulgarian,UFC Fight Night: Tuivasa vs. Tybura,03.16.2024,UFC Apex,no,Main Card,10,6,...,0,0,0,26,27,NaN,United States,United States,NaN,NaN
4,Pannie Kianzad vs. Macy Chiasson II,Macy Chiasson,Pannie Kianzad,UFC Fight Night: Tuivasa vs. Tybura,03.16.2024,UFC Apex,no,Main Card,8,16,...,7,0,0,32,32,NaN,United States,Sweden,NaN,NaN


In [7]:
#find special characters in df
winners = (df['winner'].values)
losers = (df['loser'].values)
fights = (df['fight'].values)
chars2rep = []
for winner in winners:
    if(isinstance(winner, str)):
        for char in winner:
            if char.lower() not in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ']:
                chars2rep.append(char)
            if char.upper() not in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', "Q", 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ' ']:
                chars2rep.append(char)
for loser in losers:
    if(isinstance(loser, str)):
        for char in loser:
            if char.lower() not in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ']:
                chars2rep.append(char)
            if char.upper() not in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', "Q", 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ' ']:
                chars2rep.append(char)
for fight in fights:
    if(isinstance(fight, str)):
        for char in fight:
            if char.lower() not in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ']:
                chars2rep.append(char)
            if char.upper() not in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', "Q", 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ' ']:
                chars2rep.append(char)
chars2rep = list(set(chars2rep))
for char in chars2rep:
    print(char)

In [6]:
#clean winners, losers, fights
for i in range(5):
    for index, row in df.iterrows():
        winner = row['winner']
        loser = row['loser']
        fight = row['fight']
        if(isinstance(winner, str)):
            if('ć' in winner):
                df.loc[index, 'winner'] = winner.replace('ć', 'c')
            if('ê' in winner):
                df.loc[index, 'winner'] = winner.replace('ê', 'e')
            if('Á' in winner):
                df.loc[index, 'winner'] = winner.replace('Á', 'A')
            if('Ł' in winner):
                df.loc[index, 'winner'] = winner.replace('Ł', 'L')
            if('č' in winner):
                df.loc[index, 'winner'] = winner.replace('č', 'c')
            if('ă' in winner):
                df.loc[index, 'winner'] = winner.replace('ă', 'a')
            if('á' in winner):
                df.loc[index, 'winner'] = winner.replace('á', 'a')
            if('ä' in winner):
                df.loc[index, 'winner'] = winner.replace('ä', 'a')
            if('ú' in winner):
                df.loc[index, 'winner'] = winner.replace('ú', 'u')
            if('ę' in winner):
                df.loc[index, 'winner'] = winner.replace('ę', 'e')
            if('ã' in winner):
                df.loc[index, 'winner'] = winner.replace('ã', 'a')
            if('.' in winner):
                df.loc[index, 'winner'] = winner.replace('.', '')
            if('é' in winner): 
                df.loc[index, 'winner'] = winner.replace('é', 'e')
            if('ô' in winner):
                df.loc[index, 'winner'] = winner.replace('ô', 'o')
            if("'" in winner):
                df.loc[index, 'winner'] = winner.replace("'", '')
            if('ö' in winner):
                df.loc[index, 'winner'] = winner.replace('ö', 'o')
            if('í' in winner):
                df.loc[index, 'winner'] = winner.replace('í', 'i')
            if('ř' in winner):
                df.loc[index, 'winner'] = winner.replace('ř', 'r')
            if('-' in winner):
                df.loc[index, 'winner'] = winner.replace('-', ' ')
            if('â' in winner):
                df.loc[index, 'winner'] = winner.replace('â', 'a')
            if('Ľ' in winner):
                df.loc[index, 'winner'] = winner.replace('Ľ', 'L')
            if('ţ' in winner):
                df.loc[index, 'winner'] = winner.replace('ţ', 't')
            if('ł' in winner):
                df.loc[index, 'winner'] = winner.replace('ł', 'l')
            if('õ' in winner):
                df.loc[index, 'winner'] = winner.replace('õ', 'o')
            if('š' in winner):
                df.loc[index, 'winner'] = winner.replace('š', 's')
            if('ó' in winner):
                df.loc[index, 'winner'] = winner.replace('ó', 'o')
            if('ç' in winner):
                df.loc[index, 'winner'] = winner.replace('ç', 'c')
            if('ń' in winner):
                df.loc[index, 'winner'] = winner.replace('ń', 'n')
            if('ñ' in winner):
                df.loc[index, 'winner'] = winner.replace('ñ', 'n')
            if('ž' in winner):
                df.loc[index, 'winner'] = winner.replace('ž', 'z')
        if(isinstance(loser, str)):
            if('ć' in loser):
                df.loc[index, 'loser'] = loser.replace('ć', 'c')
            if('ê' in loser):
                df.loc[index, 'loser'] = loser.replace('ê', 'e')
            if('Á' in loser):
                df.loc[index, 'loser'] = loser.replace('Á', 'A')
            if('Ł' in loser):
                df.loc[index, 'loser'] = loser.replace('Ł', 'L')
            if('č' in loser):
                df.loc[index, 'loser'] = loser.replace('č', 'c')
            if('ă' in loser):
                df.loc[index, 'loser'] = loser.replace('ă', 'a')
            if('á' in loser):
                df.loc[index, 'loser'] = loser.replace('á', 'a')
            if('ä' in loser):
                df.loc[index, 'loser'] = loser.replace('ä', 'a')
            if('ú' in loser):
                df.loc[index, 'loser'] = loser.replace('ú', 'u')
            if('ę' in loser):
                df.loc[index, 'loser'] = loser.replace('ę', 'e')
            if('ã' in loser):
                df.loc[index, 'loser'] = loser.replace('ã', 'a')
            if('.' in loser):
                df.loc[index, 'loser'] = loser.replace('.', '')
            if('é' in loser): 
                df.loc[index, 'loser'] = loser.replace('é', 'e')
            if('ô' in loser):
                df.loc[index, 'loser'] = loser.replace('ô', 'o')
            if("'" in loser):
                df.loc[index, 'loser'] = loser.replace("'", '')
            if('ö' in loser):
                df.loc[index, 'loser'] = loser.replace('ö', 'o')
            if('í' in loser):
                df.loc[index, 'loser'] = loser.replace('í', 'i')
            if('ř' in loser):
                df.loc[index, 'loser'] = loser.replace('ř', 'r')
            if('-' in loser):
                df.loc[index, 'loser'] = loser.replace('-', ' ')
            if('â' in loser):
                df.loc[index, 'loser'] = loser.replace('â', 'a')
            if('Ľ' in loser):
                df.loc[index, 'loser'] = loser.replace('Ľ', 'L')
            if('ţ' in loser):
                df.loc[index, 'loser'] = loser.replace('ţ', 't')
            if('ł' in loser):
                df.loc[index, 'loser'] = loser.replace('ł', 'l')
            if('õ' in loser):
                df.loc[index, 'loser'] = loser.replace('õ', 'o')
            if('š' in loser):
                df.loc[index, 'loser'] = loser.replace('š', 's')
            if('ó' in loser):
                df.loc[index, 'loser'] = loser.replace('ó', 'o')
            if('ç' in loser):
                df.loc[index, 'loser'] = loser.replace('ç', 'c')
            if('ń' in loser):
                df.loc[index, 'loser'] = loser.replace('ń', 'n')
            if('ñ' in loser):
                df.loc[index, 'loser'] = loser.replace('ñ', 'n')
            if('ž' in loser):
                df.loc[index, 'loser'] = loser.replace('ž', 'z')
        if(isinstance(fight, str)):
            if('ć' in fight):
                df.loc[index, 'fight'] = fight.replace('ć', 'c')
            if('ê' in fight):
                df.loc[index, 'fight'] = fight.replace('ê', 'e')
            if('Á' in fight):
                df.loc[index, 'fight'] = fight.replace('Á', 'A')
            if('Ł' in fight):
                df.loc[index, 'fight'] = fight.replace('Ł', 'L')
            if('č' in fight):
                df.loc[index, 'fight'] = fight.replace('č', 'c')
            if('ă' in fight):
                df.loc[index, 'fight'] = fight.replace('ă', 'a')
            if('á' in fight):
                df.loc[index, 'fight'] = fight.replace('á', 'a')
            if('ä' in fight):
                df.loc[index, 'fight'] = fight.replace('ä', 'a')
            if('ú' in fight):
                df.loc[index, 'fight'] = fight.replace('ú', 'u')
            if('ę' in fight):
                df.loc[index, 'fight'] = fight.replace('ę', 'e')
            if('ã' in fight):
                df.loc[index, 'fight'] = fight.replace('ã', 'a')
            if('.' in fight):
                df.loc[index, 'fight'] = fight.replace('.', '')
            if('é' in fight): 
                df.loc[index, 'fight'] = fight.replace('é', 'e')
            if('ô' in fight):
                df.loc[index, 'fight'] = fight.replace('ô', 'o')
            if("'" in fight):
                df.loc[index, 'fight'] = fight.replace("'", '')
            if('ö' in fight):
                df.loc[index, 'fight'] = fight.replace('ö', 'o')
            if('í' in fight):
                df.loc[index, 'fight'] = fight.replace('í', 'i')
            if('ř' in fight):
                df.loc[index, 'fight'] = fight.replace('ř', 'r')
            if('-' in fight):
                df.loc[index, 'fight'] = fight.replace('-', ' ')
            if('â' in fight):
                df.loc[index, 'fight'] = fight.replace('â', 'a')
            if('Ľ' in fight):
                df.loc[index, 'fight'] = fight.replace('Ľ', 'L')
            if('ţ' in fight):
                df.loc[index, 'fight'] = fight.replace('ţ', 't')
            if('ł' in fight):
                df.loc[index, 'fight'] = fight.replace('ł', 'l')
            if('õ' in fight):
                df.loc[index, 'fight'] = fight.replace('õ', 'o')
            if('š' in fight):
                df.loc[index, 'fight'] = fight.replace('š', 's')
            if('ó' in fight):
                df.loc[index, 'fight'] = fight.replace('ó', 'o')
            if('ç' in fight):
                df.loc[index, 'fight'] = fight.replace('ç', 'c')
            if('ń' in fight):
                df.loc[index, 'fight'] = fight.replace('ń', 'n')
            if('ñ' in fight):
                df.loc[index, 'fight'] = fight.replace('ñ', 'n')
            if('ž' in fight):
                df.loc[index, 'fight'] = fight.replace('ž', 'z')    
    

In [8]:
#find special characters in df2
redcorners = (df2['redCorner'].values)
bluecorners = (df2['blueCorner'].values)

chars2rep = []
for redCorner in redcorners:
    if(isinstance(redCorner, str)):
        for char in redCorner:
            if char.lower() not in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ']:
                chars2rep.append(char)
            if char.upper() not in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', "Q", 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ' ']:
                chars2rep.append(char)
for blueCorner in bluecorners:
    if(isinstance(blueCorner, str)):
        for char in blueCorner:
            if char.lower() not in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ']:
                chars2rep.append(char)
            if char.upper() not in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', "Q", 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ' ']:
                chars2rep.append(char)


chars2rep = list(set(chars2rep))
for char in chars2rep:
    print(char)

In [9]:
#clean redCorner, blueCorner, winner
for index, row in df2.iterrows():
    redCorner = row['redCorner']
    blueCorner = row['blueCorner']
    winner = row['winner']
    if(isinstance(redCorner, str)):
        if("'" in redCorner):
            df2.loc[index, 'redCorner'] = redCorner.replace("'", '')
        if('.' in redCorner):
            df2.loc[index, 'redCorner'] = redCorner.replace('.', '')
        if('-' in redCorner):
            df2.loc[index, 'redCorner'] = redCorner.replace('-', ' ')
    if(isinstance(blueCorner, str)):
        if("'" in blueCorner):
            df2.loc[index, 'blueCorner'] = blueCorner.replace("'", '')
        if('.' in blueCorner):
            df2.loc[index, 'blueCorner'] = blueCorner.replace('.', '')
        if('-' in blueCorner):
            df2.loc[index, 'blueCorner'] = blueCorner.replace('-', ' ')
    if(isinstance(winner, str)):
        if("'" in winner):
            df2.loc[index, 'winner'] = winner.replace("'", '')
        if('.' in winner):
            df2.loc[index, 'winner'] = winner.replace('.', '')
        if('-' in winner):
            df2.loc[index, 'winner'] = winner.replace('-', ' ')
    
    

In [10]:
#define headers
column_headers = [
    'fight', 'redCorner', 'blueCorner', 'winner', 'event', 'referee', 'method_of_victory',
    'date', 'venue', 'title_fight', 'billing', 'redCorner_wins', 'blueCorner_wins',
    'redCorner_losses', 'blueCorner_losses', 'redCorner_draws', 'blueCorner_draws',
    'redCorner_age', 'blueCorner_age', 'redCorner_nation', 'blueCorner_nation',
    'redCorner_fan', 'blueCorner_fan', 'redCorner_knockdowns', 'blueCorner_knockdowns',
    'redCorner_sig_str', 'blueCorner_sig_str', 'redCorner_sig_str_percentage',
    'blueCorner_sig_str_percentage', 'redCorner_total_str', 'blueCorner_total_str',
    'redCorner_takedowns', 'blueCorner_takedowns', 'redCorner_takedown_percentage',
    'blueCorner_takedown_percentage', 'redCorner_subs_attempted', 'blueCorner_subs_attempted', 'round', 'time',
    'redCorner_height', 'blueCorner_height', 'redCorner_reach', 'blueCorner_reach', 'redCorner_stance', 'blueCorner_stance'
]

#create dataframe using headers
dfNew = pd.DataFrame(columns=column_headers)

dfNew.head()

,fight,redCorner,blueCorner,winner,event,referee,method_of_victory,date,venue,title_fight,...,redCorner_subs_attempted,blueCorner_subs_attempted,round,time,redCorner_height,blueCorner_height,redCorner_reach,blueCorner_reach,redCorner_stance,blueCorner_stance


In [11]:
#fix some inconsistencies
df2['redCorner'] = df2['redCorner'].replace('Loopy Godinez', 'Lupita Godinez', regex=True)
df2['blueCorner'] = df2['blueCorner'].replace('Loopy Godinez', 'Lupita Godinez', regex=True)
df2['winner'] = df2['winner'].replace('Loopy Godinez', 'Lupita Godinez', regex=True)

df2['redCorner'] = df2['redCorner'].replace('Viktoriia Dudakova', 'Victoria Dudakova', regex=True)
df2['blueCorner'] = df2['blueCorner'].replace('Viktoriia Dudakova', 'Victoria Dudakova', regex=True)
df2['winner'] = df2['winner'].replace('Viktoriia Dudakova', 'Victoria Dudakova', regex=True)

#fight overturned - only shown in one df
index = (df['fight'] == 'Miles Johns vs Dan Argueta') & (df['date'] == '09.23.2023')
df.loc[index, 'winner'] = ''
df.loc[index, 'loser'] = ''

df2['redCorner'] = df2['redCorner'].replace('Blood Diamond', 'Mike Mathetha', regex=True)
df2['blueCorner'] = df2['blueCorner'].replace('Blood Diamond', 'Mike Mathetha', regex=True)
df2['winner'] = df2['winner'].replace('Blood Diamond', 'Mike Mathetha', regex=True)

df2['redCorner'] = df2['redCorner'].replace('Assu Almabayev', 'Asu Almabaev', regex=True)
df2['blueCorner'] = df2['blueCorner'].replace('Assu Almabayev', 'Asu Almabaev', regex=True)
df2['winner'] = df2['winner'].replace('Assu Almabayev', 'Asu Almabaev', regex=True)

df['fight'] = df['fight'].replace('Carl Deaton III', 'Carl Deaton', regex=True)
df['winner'] = df['winner'].replace('Carl Deaton III', 'Carl Deaton', regex=True)
df['loser'] = df['loser'].replace('Carl Deaton III', 'Carl Deaton', regex=True)

df2['redCorner'] = df2['redCorner'].replace('Alexander Munoz', 'Alex Munoz', regex=True)
df2['blueCorner'] = df2['blueCorner'].replace('Alexander Munoz', 'Alex Munoz', regex=True)
df2['winner'] = df2['winner'].replace('Alexander Munoz', 'Alex Munoz', regex=True)

df['fight'] = df['fight'].replace('Ovince St Preux', 'Ovince Saint Preux', regex=True)
df['winner'] = df['winner'].replace('Ovince St Preux', 'Ovince Saint Preux', regex=True)
df['loser'] = df['loser'].replace('Ovince St Preux', 'Ovince Saint Preux', regex=True)

df2['redCorner'] = df2['redCorner'].replace('Kazula Vargas', 'Rodrigo Vargas', regex=True)
df2['blueCorner'] = df2['blueCorner'].replace('Kazula Vargas', 'Rodrigo Vargas', regex=True)
df2['winner'] = df2['winner'].replace('Kazula Vargas', 'Rodrigo Vargas', regex=True)

df2['redCorner'] = df2['redCorner'].replace('Da Woon Jung', 'Da Un Jung', regex=True)
df2['blueCorner'] = df2['blueCorner'].replace('Da Woon Jung', 'Da Un Jung', regex=True)
df2['winner'] = df2['winner'].replace('Da Woon Jung', 'Da Un Jung', regex=True)

df2['redCorner'] = df2['redCorner'].replace('Lara Procopio', 'Lara Fritzen', regex=True)
df2['blueCorner'] = df2['blueCorner'].replace('Lara Procopio', 'Lara Fritzen', regex=True)
df2['winner'] = df2['winner'].replace('Lara Procopio', 'Lara Fritzen', regex=True)

df2['redCorner'] = df2['redCorner'].replace('Jacare Souza', 'Ronaldo Souza', regex=True)
df2['blueCorner'] = df2['blueCorner'].replace('Jacare Souza', 'Ronaldo Souza', regex=True)
df2['winner'] = df2['winner'].replace('Jacare Souza', 'Ronaldo Souza', regex=True)

df['fight'] = df['fight'].replace('Jose Alberto Quinonez', 'Jose Quinonez', regex=True)
df['winner'] = df['winner'].replace('Jose Alberto Quinonez', 'Jose Quinonez', regex=True)
df['loser'] = df['loser'].replace('Jose Alberto Quinonez', 'Jose Quinonez', regex=True)

df2['redCorner'] = df2['redCorner'].replace('Mara Romero Borella', 'Mara Borella', regex=True)
df2['blueCorner'] = df2['blueCorner'].replace('Mara Romero Borella', 'Mara Borella', regex=True)
df2['winner'] = df2['winner'].replace('Mara Romero Borella', 'Mara Borella', regex=True)

df2['redCorner'] = df2['redCorner'].replace('Grigory Popov', 'Grigorii Popov', regex=True)
df2['blueCorner'] = df2['blueCorner'].replace('Grigory Popov', 'Grigorii Popov', regex=True)
df2['winner'] = df2['winner'].replace('Grigory Popov', 'Grigorii Popov', regex=True)

df['fight'] = df['fight'].replace('Yanan Wu', 'Wu Yanan', regex=True)
df['winner'] = df['winner'].replace('Yanan Wu', 'Wu Yanan', regex=True)
df['loser'] = df['loser'].replace('Yanan Wu', 'Wu Yanan', regex=True)

df['fight'] = df['fight'].replace('Alexey Kunchenko', 'Aleskei Kunchenko', regex=True)
df['winner'] = df['winner'].replace('Alexey Kunchenko', 'Aleskei Kunchenko', regex=True)
df['loser'] = df['loser'].replace('Alexey Kunchenko', 'Aleskei Kunchenko', regex=True)

df2['redCorner'] = df2['redCorner'].replace('Cristiane Justino', 'Cris Cyborg', regex=True)
df2['blueCorner'] = df2['blueCorner'].replace('Cristiane Justino', 'Cris Cyborg', regex=True)
df2['winner'] = df2['winner'].replace('Cristiane Justino', 'Cris Cyborg', regex=True)

df['fight'] = df['fight'].replace('Des Green', 'Desmond Green', regex=True)
df['winner'] = df['winner'].replace('Des Green', 'Desmond Green', regex=True)
df['loser'] = df['loser'].replace('Des Green', 'Desmond Green', regex=True)

df['fight'] = df['fight'].replace('Dmitry Smolyakov', 'Dmitrii Smoliakov', regex=True)
df['winner'] = df['winner'].replace('Dmitry Smolyakov', 'Dmitrii Smoliakov', regex=True)
df['loser'] = df['loser'].replace('Dmitry Smolyakov', 'Dmitrii Smoliakov', regex=True)

df['fight'] = df['fight'].replace('Ulka Sasaki', 'Yuta Sasaki', regex=True)
df['winner'] = df['winner'].replace('Ulka Sasaki', 'Yuta Sasaki', regex=True)
df['loser'] = df['loser'].replace('Ulka Sasaki', 'Yuta Sasaki', regex=True)

df['fight'] = df['fight'].replace('Roberto Sanchez', 'Robert Sanchez', regex=True)
df['winner'] = df['winner'].replace('Roberto Sanchez', 'Robert Sanchez', regex=True)
df['loser'] = df['loser'].replace('Roberto Sanchez', 'Robert Sanchez', regex=True)

df['fight'] = df['fight'].replace('Dmitriy Sosnovskiy', 'Dmitry Sosnovskiy', regex=True)
df['winner'] = df['winner'].replace('Dmitriy Sosnovskiy', 'Dmitry Sosnovskiy', regex=True)
df['loser'] = df['loser'].replace('Dmitriy Sosnovskiy', 'Dmitry Sosnovskiy', regex=True)

df2['redCorner'] = df2['redCorner'].replace('Timothy Johnson', 'Tim Johnson', regex=True)
df2['blueCorner'] = df2['blueCorner'].replace('Timothy Johnson', 'Tim Johnson', regex=True)
df2['winner'] = df2['winner'].replace('Timothy Johnson', 'Tim Johnson', regex=True)

df['fight'] = df['fight'].replace('Maia Kahaunaele', 'Maia Stevenson', regex=True)
df['winner'] = df['winner'].replace('Maia Kahaunaele', 'Maia Stevenson', regex=True)
df['loser'] = df['loser'].replace('Maia Kahaunaele', 'Maia Stevenson', regex=True)

df['fight'] = df['fight'].replace('Bharat Khandare', 'Bharat Kandare', regex=True)
df['winner'] = df['winner'].replace('Bharat Khandare', 'Bharat Kandare', regex=True)
df['loser'] = df['loser'].replace('Bharat Khandare', 'Bharat Kandare', regex=True)

df['fight'] = df['fight'].replace('Nico Musoke', 'Nicholas Musoke', regex=True)
df['winner'] = df['winner'].replace('Nico Musoke', 'Nicholas Musoke', regex=True)
df['loser'] = df['loser'].replace('Nico Musoke', 'Nicholas Musoke', regex=True)

#fight overturned
index = (df['fight'] == 'Alex Morono vs Niko Price') & (df['date'] == '02.04.2017')
df.loc[index, 'winner'] = ''
df.loc[index, 'loser'] = ''

df2['redCorner'] = df2['redCorner'].replace('Joseph Gigliotti', 'Joe Gigliotti', regex=True)
df2['blueCorner'] = df2['blueCorner'].replace('Joseph Gigliotti', 'Joe Gigliotti', regex=True)
df2['winner'] = df2['winner'].replace('Joseph Gigliotti', 'Joe Gigliotti', regex=True)

df2['redCorner'] = df2['redCorner'].replace('Tiago dos Santos e Silva', 'Tiago Trator', regex=True)
df2['blueCorner'] = df2['blueCorner'].replace('Tiago dos Santos e Silva', 'Tiago Trator', regex=True)
df2['winner'] = df2['winner'].replace('Tiago dos Santos e Silva', 'Tiago Trator', regex=True)

df['fight'] = df['fight'].replace('Manny Gamburyan', 'Manvel Gamburyan', regex=True)
df['winner'] = df['winner'].replace('Manny Gamburyan', 'Manvel Gamburyan', regex=True)
df['loser'] = df['loser'].replace('Manny Gamburyan', 'Manvel Gamburyan', regex=True)

df['fight'] = df['fight'].replace('Mike Graves', 'Michael Graves', regex=True)
df['winner'] = df['winner'].replace('Mike Graves', 'Michael Graves', regex=True)
df['loser'] = df['loser'].replace('Mike Graves', 'Michael Graves', regex=True)

df['fight'] = df['fight'].replace('Marcio Alexandre Jr', 'Marcio Alexandre Junior', regex=True)
df['winner'] = df['winner'].replace('Marcio Alexandre Jr', 'Marcio Alexandre Junior', regex=True)
df['loser'] = df['loser'].replace('Marcio Alexandre Jr', 'Marcio Alexandre Junior', regex=True)

df['fight'] = df['fight'].replace('Steven Kennedy', 'Steve Kennedy', regex=True)
df['winner'] = df['winner'].replace('Steven Kennedy', 'Steve Kennedy', regex=True)
df['loser'] = df['loser'].replace('Steven Kennedy', 'Steve Kennedy', regex=True)

df['fight'] = df['fight'].replace('Ronald Stallings', 'Ron Stallings', regex=True)
df['winner'] = df['winner'].replace('Ronald Stallings', 'Ron Stallings', regex=True)
df['loser'] = df['loser'].replace('Ronald Stallings', 'Ron Stallings', regex=True)

df['fight'] = df['fight'].replace('Tony Christodoulou', 'Anthony Christodoulou', regex=True)
df['winner'] = df['winner'].replace('Tony Christodoulou', 'Anthony Christodoulou', regex=True)
df['loser'] = df['loser'].replace('Tony Christodoulou', 'Anthony Christodoulou', regex=True)

df['fight'] = df['fight'].replace('Costas Philippou', 'Constantinos Philippou', regex=True)
df['winner'] = df['winner'].replace('Costas Philippou', 'Constantinos Philippou', regex=True)
df['loser'] = df['loser'].replace('Costas Philippou', 'Constantinos Philippou', regex=True)

df['fight'] = df['fight'].replace('Alp Ozkilic', 'Alptekin Ozkilic', regex=True)
df['winner'] = df['winner'].replace('Alp Ozkilic', 'Alptekin Ozkilic', regex=True)
df['loser'] = df['loser'].replace('Alp Ozkilic', 'Alptekin Ozkilic', regex=True)

df['fight'] = df['fight'].replace('Robbie Peralta', 'Robert Peralta', regex=True)
df['winner'] = df['winner'].replace('Robbie Peralta', 'Robert Peralta', regex=True)
df['loser'] = df['loser'].replace('Robbie Peralta', 'Robert Peralta', regex=True)

df2['redCorner'] = df2['redCorner'].replace('Elizeu Zaleski dos Santos', 'Elizeu Zaleski', regex=True)
df2['blueCorner'] = df2['blueCorner'].replace('Elizeu Zaleski dos Santos', 'Elizeu Zaleski', regex=True)
df2['winner'] = df2['winner'].replace('Elizeu Zaleski dos Santos', 'Elizeu Zaleski', regex=True)

#fight overturned
index = (df['fight'] == 'Norifumi Yamamoto vs Roman Salazar') & (df['date'] == '02.28.2015')
df.loc[index, 'winner'] = ''
df.loc[index, 'loser'] = ''

df['fight'] = df['fight'].replace('Alexander Torres', 'Alex Torres', regex=True)
df['winner'] = df['winner'].replace('Alexander Torres', 'Alex Torres', regex=True)
df['loser'] = df['loser'].replace('Alexander Torres', 'Alex Torres', regex=True)

df['fight'] = df['fight'].replace('Pat Walsh', 'Patrick Walsh', regex=True)
df['winner'] = df['winner'].replace('Pat Walsh', 'Patrick Walsh', regex=True)
df['loser'] = df['loser'].replace('Pat Walsh', 'Patrick Walsh', regex=True)

df['fight'] = df['fight'].replace('Zhumabek Tursyn', 'Jumabieke Tuerxun', regex=True)
df['winner'] = df['winner'].replace('Zhumabek Tursyn', 'Jumabieke Tuerxun', regex=True)
df['loser'] = df['loser'].replace('Zhumabek Tursyn', 'Jumabieke Tuerxun', regex=True)

df['fight'] = df['fight'].replace('Dan Spohn', 'Daniel Spohn', regex=True)
df['winner'] = df['winner'].replace('Dan Spohn', 'Daniel Spohn', regex=True)
df['loser'] = df['loser'].replace('Dan Spohn', 'Daniel Spohn', regex=True)

df['fight'] = df['fight'].replace('Guilherme Bomba', 'Guilherme Vasconcelos', regex=True)
df['winner'] = df['winner'].replace('Guilherme Bomba', 'Guilherme Vasconcelos', regex=True)
df['loser'] = df['loser'].replace('Guilherme Bomba', 'Guilherme Vasconcelos', regex=True)

df['fight'] = df['fight'].replace('Bubba McDaniel', 'Robert McDaniel', regex=True)
df['winner'] = df['winner'].replace('Bubba McDaniel', 'Robert McDaniel', regex=True)
df['loser'] = df['loser'].replace('Bubba McDaniel', 'Robert McDaniel', regex=True)

#fight overturned
index = (df['fight'] == 'Louis Gaudinot vs Phil Harris') & (df['date'] == '03.08.2014')
df.loc[index, 'winner'] = ''
df.loc[index, 'loser'] = ''

df['fight'] = df['fight'].replace('Benny Alloway', 'Ben Alloway', regex=True)
df['winner'] = df['winner'].replace('Benny Alloway', 'Ben Alloway', regex=True)
df['loser'] = df['loser'].replace('Benny Alloway', 'Ben Alloway', regex=True)

df['fight'] = df['fight'].replace('Phil De Fries', 'Philip De Fries', regex=True)
df['winner'] = df['winner'].replace('Phil De Fries', 'Philip De Fries', regex=True)
df['loser'] = df['loser'].replace('Phil De Fries', 'Philip De Fries', regex=True)

df['fight'] = df['fight'].replace('Matt Riddle', 'Matthew Riddle', regex=True)
df['winner'] = df['winner'].replace('Matt Riddle', 'Matthew Riddle', regex=True)
df['loser'] = df['loser'].replace('Matt Riddle', 'Matthew Riddle', regex=True)

df['fight'] = df['fight'].replace('Manny Rodriguez', 'Manuel Rodriguez', regex=True)
df['winner'] = df['winner'].replace('Manny Rodriguez', 'Manuel Rodriguez', regex=True)
df['loser'] = df['loser'].replace('Manny Rodriguez', 'Manuel Rodriguez', regex=True)

df['fight'] = df['fight'].replace('John Olav Einemo', 'Jon Olav Einemo', regex=True)
df['winner'] = df['winner'].replace('John Olav Einemo', 'Jon Olav Einemo', regex=True)
df['loser'] = df['loser'].replace('John Olav Einemo', 'Jon Olav Einemo', regex=True)

df['fight'] = df['fight'].replace('Kimbo Slice', 'Kevin Ferguson', regex=True)
df['winner'] = df['winner'].replace('Kimbo Slice', 'Kevin Ferguson', regex=True)
df['loser'] = df['loser'].replace('Kimbo Slice', 'Kevin Ferguson', regex=True)

df['fight'] = df['fight'].replace('Roli Delgado', 'Rolando Delgado', regex=True)
df['winner'] = df['winner'].replace('Roli Delgado', 'Rolando Delgado', regex=True)
df['loser'] = df['loser'].replace('Roli Delgado', 'Rolando Delgado', regex=True)

df['fight'] = df['fight'].replace('Dave Kaplan', 'David Kaplan', regex=True)
df['winner'] = df['winner'].replace('Dave Kaplan', 'David Kaplan', regex=True)
df['loser'] = df['loser'].replace('Dave Kaplan', 'David Kaplan', regex=True)

df['fight'] = df['fight'].replace('Mike Patt', 'Michael Patt', regex=True)
df['winner'] = df['winner'].replace('Mike Patt', 'Michael Patt', regex=True)
df['loser'] = df['loser'].replace('Mike Patt', 'Michael Patt', regex=True)

df['fight'] = df['fight'].replace('Thomas Speer', 'Tommy Speer', regex=True)
df['winner'] = df['winner'].replace('Thomas Speer', 'Tommy Speer', regex=True)
df['loser'] = df['loser'].replace('Thomas Speer', 'Tommy Speer', regex=True)

df['fight'] = df['fight'].replace('Douglas Evans', 'Doug Evans', regex=True)
df['winner'] = df['winner'].replace('Douglas Evans', 'Doug Evans', regex=True)
df['loser'] = df['loser'].replace('Douglas Evans', 'Doug Evans', regex=True)

df['fight'] = df['fight'].replace('Daniel Barrera', 'Dan Barrera', regex=True)
df['winner'] = df['winner'].replace('Daniel Barrera', 'Dan Barrera', regex=True)
df['loser'] = df['loser'].replace('Daniel Barrera', 'Dan Barrera', regex=True)

df['fight'] = df['fight'].replace('Allen Berubie', 'Allen Berube', regex=True)
df['winner'] = df['winner'].replace('Allen Berubie', 'Allen Berube', regex=True)
df['loser'] = df['loser'].replace('Allen Berubie', 'Allen Berube', regex=True)

df['fight'] = df['fight'].replace('Yoshitomi Mishima', 'Dokonjonosuke Mishima', regex=True)
df['winner'] = df['winner'].replace('Yoshitomi Mashima', 'Dokonjonosuke Mashima', regex=True)
df['loser'] = df['loser'].replace('Yoshitomi Mashima', 'Dokonjonosuke Mashima', regex=True)

df2['redCorner'] = df2['redCorner'].replace('Steve Lynch', 'Steven Lynch', regex=True)
df2['blueCorner'] = df2['blueCorner'].replace('Steve Lynch', 'Steven Lynch', regex=True)
df2['winner'] = df2['winner'].replace('Steve Lynch', 'Steven Lynch', regex=True)

df2['redCorner'] = df2['redCorner'].replace('Stevie Lynch', 'Steven Lynch', regex=True)
df2['blueCorner'] = df2['blueCorner'].replace('Stevie Lynch', 'Steven Lynch', regex=True)
df2['winner'] = df2['winner'].replace('Stevie Lynch', 'Steven Lynch', regex=True)

df['fight'] = df['fight'].replace('Josh Schockman', 'Josh Shockman', regex=True)
df['winner'] = df['winner'].replace('Josh Schockman', 'Josh Shockman', regex=True)
df['loser'] = df['loser'].replace('Josh Schockman', 'Josh Shockman', regex=True)

df2['redCorner'] = df2['redCorner'].replace('Sammy Morgan', 'Sam Morgan', regex=True)
df2['blueCorner'] = df2['blueCorner'].replace('Sammy Morgan', 'Sam Morgan', regex=True)
df2['winner'] = df2['winner'].replace('Sammy Morgan', 'Sam Morgan', regex=True)

df['fight'] = df['fight'].replace('Kris Rotharmel', 'Kristian Rothaermel', regex=True)
df['winner'] = df['winner'].replace('Kris Rotharmel', 'Kristian Rothaermel', regex=True)
df['loser'] = df['loser'].replace('Kris Rotharmel', 'Kristian Rothaermel', regex=True)

df['fight'] = df['fight'].replace('Joao Marcos Pierini', 'Joao Pierini', regex=True)
df['winner'] = df['winner'].replace('Joao Marcos Pierini', 'Joao Pierini', regex=True)
df['loser'] = df['loser'].replace('Joao Marcos Pierini', 'Joao Pierini', regex=True)

df['fight'] = df['fight'].replace('Tsuyoshi Kosaka', 'Tsuyoshi Kohsaka', regex=True)
df['winner'] = df['winner'].replace('Tsuyoshi Kosaka', 'Tsuyoshi Kohsaka', regex=True)
df['loser'] = df['loser'].replace('Tsuyoshi Kosaka', 'Tsuyoshi Kohsaka', regex=True)

df['fight'] = df['fight'].replace('Andrey Semenov', 'Andrei Semenov', regex=True)
df['winner'] = df['winner'].replace('Andrey Semenov', 'Andrei Semenov', regex=True)
df['loser'] = df['loser'].replace('Andrey Semenov', 'Andrei Semenov', regex=True)

df2['redCorner'] = df2['redCorner'].replace('Cesar Marsucci', 'Cesar Marscucci', regex=True)
df2['blueCorner'] = df2['blueCorner'].replace('Cesar Marsucci', 'Cesar Marscucci', regex=True)
df2['winner'] = df2['winner'].replace('Cesar Marsucci', 'Cesar Marscucci', regex=True)

df['fight'] = df['fight'].replace('Cristophe Leninger', 'Christophe Leninger', regex=True)
df['winner'] = df['winner'].replace('Cristophe Leninger', 'Christophe Leninger', regex=True)
df['loser'] = df['loser'].replace('Cristophe Leninger', 'Christophe Leninger', regex=True)

df['fight'] = df['fight'].replace('Kazuo Takahashi', 'Yoshiki Takahashi', regex=True)
df['winner'] = df['winner'].replace('Kazuo Takahashi', 'Yoshiki Takahashi', regex=True)
df['loser'] = df['loser'].replace('Kazuo Takahashi', 'Yoshiki Takahashi', regex=True)

df2['redCorner'] = df2['redCorner'].replace('Felix Lee Mitchell', 'Felix Mitchell', regex=True)
df2['blueCorner'] = df2['blueCorner'].replace('Felix Lee Mitchell', 'Felix Mitchell', regex=True)
df2['winner'] = df2['winner'].replace('Felix Lee Mitchell', 'Felix Mitchell', regex=True)

df['fight'] = df['fight'].replace('John Campatella', 'John Campetella', regex=True)
df['winner'] = df['winner'].replace('John Campatella', 'John Campetella', regex=True)
df['loser'] = df['loser'].replace('John Campatella', 'John Campetella', regex=True)

df['fight'] = df['fight'].replace('Eldo Dias Xavier', 'Eldo Xavier Diaz', regex=True)
df['winner'] = df['winner'].replace('Eldo Dias Xavier', 'Eldo Xavier Diaz', regex=True)
df['loser'] = df['loser'].replace('Eldo Dias Xavier', 'Eldo Xavier Diaz', regex=True)

df['fight'] = df['fight'].replace('Alberto Cerra Leon', 'Alberta Cerra Leon', regex=True)
df['winner'] = df['winner'].replace('Alberto Cerra Leon', 'Alberta Cerra Leon', regex=True)
df['loser'] = df['loser'].replace('Alberto Cerra Leon', 'Alberta Cerra Leon', regex=True)

In [26]:
df2.head()

,redCorner,blueCorner,winner,event,referee,method_of_victory,red_Knockdowns,blue_Knockdowns,red_sig_str,blue_sig_str,...,red_subs_attempted,blue_subs_attempted,round,time,redCorner_height,blueCorner_height,redCorner_reach,blueCorner_reach,redCorner_stance,blueCorner_stance
0,Tai Tuivasa,Marcin Tybura,Marcin Tybura,UFC Fight Night: Tuivasa vs. Tybura,Herb Dean,Submission,0,0,15 of 24,27 of 41,...,0,1,1,4:08,"6' 2""","6' 3""","75""","78""",Southpaw,Orthodox
1,Bryan Battle,Ange Loosa,NaN,UFC Fight Night: Tuivasa vs. Tybura,Mike Beltran,Could Not Continue,0,0,27 of 75,23 of 54,...,0,0,2,1:00,"6' 1""","5' 10""","77""","74""",Orthodox,Orthodox
2,Ovince Saint Preux,Kennedy Nzechukwu,Ovince Saint Preux,UFC Fight Night: Tuivasa vs. Tybura,Mark Smith,Decision - Split,1,0,143 of 276,106 of 250,...,0,0,3,5:00,"6' 3""","6' 5""","80""","83""",Southpaw,Southpaw
3,Christian Rodriguez,Isaac Dulgarian,Christian Rodriguez,UFC Fight Night: Tuivasa vs. Tybura,Chris Tognoni,Decision - Split,0,0,48 of 76,22 of 44,...,0,2,3,5:00,"5' 7""","5' 7""","71""","71""",Orthodox,Orthodox
4,Pannie Kianzad,Macy Chiasson,Macy Chiasson,UFC Fight Night: Tuivasa vs. Tybura,Mark Smith,Submission,0,0,2 of 6,4 of 10,...,0,1,1,3:54,"5' 7""","5' 11""","66""","72""",Orthodox,Orthodox


In [27]:
for (index, row), (index2, row2) in zip(df.iterrows(), df2.iterrows()):
    if(row['event'] == row2['event']):
        fight = row['fight'].split(' vs ')
        fighter1 = ''.join(sorted(str(fight[0]).replace(" ", "").lower()))
        fighter2 = ''.join(sorted(str(fight[1]).replace(" ", "").lower()))
        redCorner = ''.join(sorted(str(row2['redCorner']).replace(" ", "").lower()))
        blueCorner = ''.join(sorted(str(row2['blueCorner']).replace(" ", "").lower()))
        winner1 = ''.join(sorted(str(row['winner']).replace(" ", "").lower()))
        winner2 = ''.join(sorted(str(row2['winner']).replace(" ", "").lower()))
        if(fighter1 in redCorner or fighter1 in blueCorner or redCorner in fighter1 or blueCorner in fighter1):
            if(fighter2 in redCorner or fighter2 in blueCorner or redCorner in fighter2 or blueCorner in fighter2):
                if(winner1 in winner2 or winner2 in winner1):
                    fight = row['fight']
                    redCorner = row2['redCorner']
                    blueCorner = row2['blueCorner']
                    winner = row2['winner']
                    event = row2['event']
                    referee = row2['referee']
                    method_of_vic = row2['method_of_victory']
                    date = row['date']
                    venue = row['venue']
                    title_fight = row['title_fight']
                    billing = row['billing']
                    if(winner1 in redCorner or redCorner in winner1):
                        redCorner_wins = row['winner_wins']
                        redCorner_losses = row['winner_losses']
                        redCorner_draws = row['winner_draws']
                        redCorner_age = row['winner_age']
                        redCorner_nation = row['winner_nationality']
                        redCorner_fan = row['winner_fan ']
                        blueCorner_wins = row['loser_wins']
                        blueCorner_losses = row['loser_losses']
                        blueCorner_draws = row['loser_draws']
                        blueCorner_age = row['loser_age']
                        blueCorner_nation = row['loser_nationality']
                        blueCorner_fan = row['loser_fan']
                    elif(winner1 in blueCorner or blueCorner in winner1):
                        blueCorner_wins = row['winner_wins']
                        blueCorner_losses = row['winner_losses']
                        blueCorner_draws = row['winner_draws']
                        blueCorner_age = row['winner_age']
                        blueCorner_nation = row['winner_nationality']
                        blueCorner_fan = row['winner_fan ']
                        redCorner_wins = row['loser_wins']
                        redCorner_losses = row['loser_losses']
                        redCorner_draws = row['loser_draws']
                        redCorner_age = row['loser_age']
                        redCorner_nation = row['loser_nationality']
                        redCorner_fan = row['loser_fan']
                    else:
                        if(fight[0] in redCorner or redCorner in fight[0]):
                            redCorner_wins = row['winner_wins']
                            redCorner_losses = row['winner_losses']
                            redCorner_draws = row['winner_draws']
                            redCorner_age = row['winner_age']
                            redCorner_nation = row['winner_nationality']
                            redCorner_fan = row['winner_fan ']
                            blueCorner_wins = row['loser_wins']
                            blueCorner_losses = row['loser_losses']
                            blueCorner_draws = row['loser_draws']
                            blueCorner_age = row['loser_age']
                            blueCorner_nation = row['loser_nationality']
                            blueCorner_fan = row['loser_fan']
                        if(fight[0] in blueCorner or blueCorner in fight[0]):
                            blueCorner_wins = row['winner_wins']
                            blueCorner_losses = row['winner_losses']
                            blueCorner_draws = row['winner_draws']
                            blueCorner_age = row['winner_age']
                            blueCorner_nation = row['winner_nationality']
                            blueCorner_fan = row['winner_fan ']
                            redCorner_wins = row['loser_wins']
                            redCorner_losses = row['loser_losses']
                            redCorner_draws = row['loser_draws']
                            redCorner_age = row['loser_age']
                            redCorner_nation = row['loser_nationality']
                            redCorner_fan = row['loser_fan']
                    redCorner_knockdowns = row2['red_Knockdowns']
                    blueCorner_knockdowns = row2['blue_Knockdowns']
                    redCorner_sig_str = row2['red_sig_str']
                    blueCorner_sig_str = row2['blue_sig_str']
                    redCorner_sig_str_percentage = row2['red_sig_str_percentage']
                    blueCorner_sig_str_percentage = row2['blue_sig_str_percentage']
                    redCorner_total_str = row2['red_total_strikes']
                    blueCorner_total_str = row2['blue_total_strikes']
                    redCorner_takedowns = row2['red_takedowns']
                    blueCorner_takedowns = row2['blue_takedowns']
                    redCorner_takedown_percentage = row2['red_takedown_percentage']
                    blueCorner_takedown_percentage = row2['blue_takedown_percentage']
                    redCorner_subs_attempted = row2['red_subs_attempted']
                    blueCorner_subs_attempted = row2['blue_subs_attempted']
                    roundA = row2['round']      
                    time = row2['time']
                    redCorner_height = row2['redCorner_height']
                    blueCorner_height = row2['blueCorner_height']
                    redCorner_reach = row2['redCorner_reach']
                    blueCorner_reach = row2['blueCorner_reach']
                    redCorner_stance = row2['redCorner_stance']
                    blueCorner_stance = row2['blueCorner_stance']
            else:
                fight = row['fight'].split(' vs ')
                fighter1 = str(fight[0]).replace(" ", "").lower()
                fighter2 = str(fight[1]).replace(" ", "").lower()
                redCorner = str(row2['redCorner']).replace(" ", "").lower()
                blueCorner = str(row2['blueCorner']).replace(" ", "").lower()
                winner1 = str(row['winner']).replace(" ", "").lower()
                winner2 = str(row2['winner']).replace(" ", "").lower()
                if(fighter1 in redCorner or fighter1 in blueCorner or redCorner in fighter1 or blueCorner in fighter1):
                    if(fighter2 in redCorner or fighter2 in blueCorner or redCorner in fighter2 or blueCorner in fighter2):
                        if(winner1 in winner2 or winner2 in winner1):
                            fight = row['fight']
                            redCorner = row2['redCorner']
                            blueCorner = row2['blueCorner']
                            winner = row2['winner']
                            event = row2['event']
                            referee = row2['referee']
                            method_of_vic = row2['method_of_victory']
                            date = row['date']
                            venue = row['venue']
                            title_fight = row['title_fight']
                            billing = row['billing']
                            if(winner1 in redCorner or redCorner in winner1):
                                redCorner_wins = row['winner_wins']
                                redCorner_losses = row['winner_losses']
                                redCorner_draws = row['winner_draws']
                                redCorner_age = row['winner_age']
                                redCorner_nation = row['winner_nationality']
                                redCorner_fan = row['winner_fan ']
                                blueCorner_wins = row['loser_wins']
                                blueCorner_losses = row['loser_losses']
                                blueCorner_draws = row['loser_draws']
                                blueCorner_age = row['loser_age']
                                blueCorner_nation = row['loser_nationality']
                                blueCorner_fan = row['loser_fan']
                            elif(winner1 in blueCorner or blueCorner in winner1):
                                blueCorner_wins = row['winner_wins']
                                blueCorner_losses = row['winner_losses']
                                blueCorner_draws = row['winner_draws']
                                blueCorner_age = row['winner_age']
                                blueCorner_nation = row['winner_nationality']
                                blueCorner_fan = row['winner_fan ']
                                redCorner_wins = row['loser_wins']
                                redCorner_losses = row['loser_losses']
                                redCorner_draws = row['loser_draws']
                                redCorner_age = row['loser_age']
                                redCorner_nation = row['loser_nationality']
                                redCorner_fan = row['loser_fan']
                            else:
                                if(fight[0] in redCorner or redCorner in fight[0]):
                                    redCorner_wins = row['winner_wins']
                                    redCorner_losses = row['winner_losses']
                                    redCorner_draws = row['winner_draws']
                                    redCorner_age = row['winner_age']
                                    redCorner_nation = row['winner_nationality']
                                    redCorner_fan = row['winner_fan ']
                                    blueCorner_wins = row['loser_wins']
                                    blueCorner_losses = row['loser_losses']
                                    blueCorner_draws = row['loser_draws']
                                    blueCorner_age = row['loser_age']
                                    blueCorner_nation = row['loser_nationality']
                                    blueCorner_fan = row['loser_fan']
                                if(fight[0] in blueCorner or blueCorner in fight[0]):
                                    blueCorner_wins = row['winner_wins']
                                    blueCorner_losses = row['winner_losses']
                                    blueCorner_draws = row['winner_draws']
                                    blueCorner_age = row['winner_age']
                                    blueCorner_nation = row['winner_nationality']
                                    blueCorner_fan = row['winner_fan ']
                                    redCorner_wins = row['loser_wins']
                                    redCorner_losses = row['loser_losses']
                                    redCorner_draws = row['loser_draws']
                                    redCorner_age = row['loser_age']
                                    redCorner_nation = row['loser_nationality']
                                    redCorner_fan = row['loser_fan']
                            redCorner_knockdowns = row2['red_Knockdowns']
                            blueCorner_knockdowns = row2['blue_Knockdowns']
                            redCorner_sig_str = row2['red_sig_str']
                            blueCorner_sig_str = row2['blue_sig_str']
                            redCorner_sig_str_percentage = row2['red_sig_str_percentage']
                            blueCorner_sig_str_percentage = row2['blue_sig_str_percentage']
                            redCorner_total_str = row2['red_total_strikes']
                            blueCorner_total_str = row2['blue_total_strikes']
                            redCorner_takedowns = row2['red_takedowns']
                            blueCorner_takedowns = row2['blue_takedowns']
                            redCorner_takedown_percentage = row2['red_takedown_percentage']
                            blueCorner_takedown_percentage = row2['blue_takedown_percentage']
                            redCorner_subs_attempted = row2['red_subs_attempted']
                            blueCorner_subs_attempted = row2['blue_subs_attempted']
                            roundA = row2['round']      
                            time = row2['time']
                            redCorner_height = row2['redCorner_height']
                            blueCorner_height = row2['blueCorner_height']
                            redCorner_reach = row2['redCorner_reach']
                            blueCorner_reach = row2['blueCorner_reach']
                            redCorner_stance = row2['redCorner_stance']
                            blueCorner_stance = row2['blueCorner_stance'] 
        else:
            fight = row['fight'].split(' vs ')
            fighter1 = str(fight[0]).replace(" ", "").lower()
            fighter2 = str(fight[1]).replace(" ", "").lower()
            redCorner = str(row2['redCorner']).replace(" ", "").lower()
            blueCorner = str(row2['blueCorner']).replace(" ", "").lower()
            winner1 = str(row['winner']).replace(" ", "").lower()
            winner2 = str(row2['winner']).replace(" ", "").lower()
            if(fighter1 in redCorner or fighter1 in blueCorner or redCorner in fighter1 or blueCorner in fighter1):
                if(fighter2 in redCorner or fighter2 in blueCorner or redCorner in fighter2 or blueCorner in fighter2):
                    if(winner1 in winner2 or winner2 in winner1):
                        fight = row['fight']
                        redCorner = row2['redCorner']
                        blueCorner = row2['blueCorner']
                        winner = row2['winner']
                        event = row2['event']
                        referee = row2['referee']
                        method_of_vic = row2['method_of_victory']
                        date = row['date']
                        venue = row['venue']
                        title_fight = row['title_fight']
                        billing = row['billing']
                        if(winner1 in redCorner or redCorner in winner1):
                            redCorner_wins = row['winner_wins']
                            redCorner_losses = row['winner_losses']
                            redCorner_draws = row['winner_draws']
                            redCorner_age = row['winner_age']
                            redCorner_nation = row['winner_nationality']
                            redCorner_fan = row['winner_fan ']
                            blueCorner_wins = row['loser_wins']
                            blueCorner_losses = row['loser_losses']
                            blueCorner_draws = row['loser_draws']
                            blueCorner_age = row['loser_age']
                            blueCorner_nation = row['loser_nationality']
                            blueCorner_fan = row['loser_fan']
                        elif(winner1 in blueCorner or blueCorner in winner1):
                            blueCorner_wins = row['winner_wins']
                            blueCorner_losses = row['winner_losses']
                            blueCorner_draws = row['winner_draws']
                            blueCorner_age = row['winner_age']
                            blueCorner_nation = row['winner_nationality']
                            blueCorner_fan = row['winner_fan ']
                            redCorner_wins = row['loser_wins']
                            redCorner_losses = row['loser_losses']
                            redCorner_draws = row['loser_draws']
                            redCorner_age = row['loser_age']
                            redCorner_nation = row['loser_nationality']
                            redCorner_fan = row['loser_fan']
                        else:
                            if(fight[0] in redCorner or redCorner in fight[0]):
                                redCorner_wins = row['winner_wins']
                                redCorner_losses = row['winner_losses']
                                redCorner_draws = row['winner_draws']
                                redCorner_age = row['winner_age']
                                redCorner_nation = row['winner_nationality']
                                redCorner_fan = row['winner_fan ']
                                blueCorner_wins = row['loser_wins']
                                blueCorner_losses = row['loser_losses']
                                blueCorner_draws = row['loser_draws']
                                blueCorner_age = row['loser_age']
                                blueCorner_nation = row['loser_nationality']
                                blueCorner_fan = row['loser_fan']
                            if(fight[0] in blueCorner or blueCorner in fight[0]):
                                blueCorner_wins = row['winner_wins']
                                blueCorner_losses = row['winner_losses']
                                blueCorner_draws = row['winner_draws']
                                blueCorner_age = row['winner_age']
                                blueCorner_nation = row['winner_nationality']
                                blueCorner_fan = row['winner_fan ']
                                redCorner_wins = row['loser_wins']
                                redCorner_losses = row['loser_losses']
                                redCorner_draws = row['loser_draws']
                                redCorner_age = row['loser_age']
                                redCorner_nation = row['loser_nationality']
                                redCorner_fan = row['loser_fan']
                        redCorner_knockdowns = row2['red_Knockdowns']
                        blueCorner_knockdowns = row2['blue_Knockdowns']
                        redCorner_sig_str = row2['red_sig_str']
                        blueCorner_sig_str = row2['blue_sig_str']
                        redCorner_sig_str_percentage = row2['red_sig_str_percentage']
                        blueCorner_sig_str_percentage = row2['blue_sig_str_percentage']
                        redCorner_total_str = row2['red_total_strikes']
                        blueCorner_total_str = row2['blue_total_strikes']
                        redCorner_takedowns = row2['red_takedowns']
                        blueCorner_takedowns = row2['blue_takedowns']
                        redCorner_takedown_percentage = row2['red_takedown_percentage']
                        blueCorner_takedown_percentage = row2['blue_takedown_percentage']
                        redCorner_subs_attempted = row2['red_subs_attempted']
                        blueCorner_subs_attempted = row2['blue_subs_attempted']
                        roundA = row2['round']      
                        time = row2['time'] 
                        redCorner_height = row2['redCorner_height']
                        blueCorner_height = row2['blueCorner_height']
                        redCorner_reach = row2['redCorner_reach']
                        blueCorner_reach = row2['blueCorner_reach']
                        redCorner_stance = row2['redCorner_stance']
                        blueCorner_stance = row2['blueCorner_stance']
    column_vals = {
        'fight': fight,
        'redCorner': redCorner,
        'blueCorner': blueCorner,
        'winner': winner,
        'event': event,
        'referee': referee,
        'method_of_victory': method_of_vic,
        'date': date,
        'venue': venue,
        'title_fight': title_fight,
        'billing': billing,
        'redCorner_wins': redCorner_wins,
        'blueCorner_wins': blueCorner_wins,
        'redCorner_losses': redCorner_losses,
        'blueCorner_losses': blueCorner_losses,
        'redCorner_draws': redCorner_draws,
        'blueCorner_draws': blueCorner_draws,
        'redCorner_age': redCorner_age,
        'blueCorner_age': blueCorner_age,
        'redCorner_nation': redCorner_nation,
        'blueCorner_nation': blueCorner_nation,
        'redCorner_fan': redCorner_fan,
        'blueCorner_fan': blueCorner_fan,
        'redCorner_knockdowns': redCorner_knockdowns,
        'blueCorner_knockdowns': blueCorner_knockdowns,
        'redCorner_sig_str': redCorner_sig_str,
        'blueCorner_sig_str': blueCorner_sig_str,
        'redCorner_sig_str_percentage': redCorner_sig_str_percentage,
        'blueCorner_sig_str_percentage': blueCorner_sig_str_percentage,
        'redCorner_total_str': redCorner_total_str,
        'blueCorner_total_str': blueCorner_total_str,
        'redCorner_takedowns': redCorner_takedowns,
        'blueCorner_takedowns': blueCorner_takedowns,
        'redCorner_takedown_percentage': redCorner_takedown_percentage,
        'blueCorner_takedown_percentage': blueCorner_takedown_percentage,
        'redCorner_subs_attempted': redCorner_subs_attempted,
        'blueCorner_subs_attempted': blueCorner_subs_attempted,
        'round': roundA,
        'time': time,
        'redCorner_height': redCorner_height,
        'blueCorner_height': blueCorner_height,
        'redCorner_reach': redCorner_reach,
        'blueCorner_reach': blueCorner_reach,
        'redCorner_stance': redCorner_stance,
        'blueCorner_stance': blueCorner_stance
    }
    dfNew.loc[len(dfNew)] = column_vals

In [28]:
#write df to csv
dfNew.head()

,fight,redCorner,blueCorner,winner,event,referee,method_of_victory,date,venue,title_fight,...,redCorner_subs_attempted,blueCorner_subs_attempted,round,time,redCorner_height,blueCorner_height,redCorner_reach,blueCorner_reach,redCorner_stance,blueCorner_stance
0,Tai Tuivasa vs Marcin Tybura,Tai Tuivasa,Marcin Tybura,Marcin Tybura,UFC Fight Night: Tuivasa vs. Tybura,Herb Dean,Submission,03.16.2024,UFC Apex,no,...,0,1,1,4:08,"6' 2""","6' 3""","75""","78""",Southpaw,Orthodox
1,Bryan Battle vs Ange Loosa,Bryan Battle,Ange Loosa,NaN,UFC Fight Night: Tuivasa vs. Tybura,Mike Beltran,Could Not Continue,03.16.2024,UFC Apex,no,...,0,0,2,1:00,"6' 1""","5' 10""","77""","74""",Orthodox,Orthodox
2,Kennedy Nzechukwu vs Ovince Saint Preux,Ovince Saint Preux,Kennedy Nzechukwu,Ovince Saint Preux,UFC Fight Night: Tuivasa vs. Tybura,Mark Smith,Decision - Split,03.16.2024,UFC Apex,no,...,0,0,3,5:00,"6' 3""","6' 5""","80""","83""",Southpaw,Southpaw
3,Christian Rodriguez vs Isaac Dulgarian,Christian Rodriguez,Isaac Dulgarian,Christian Rodriguez,UFC Fight Night: Tuivasa vs. Tybura,Chris Tognoni,Decision - Split,03.16.2024,UFC Apex,no,...,0,2,3,5:00,"5' 7""","5' 7""","71""","71""",Orthodox,Orthodox
4,Pannie Kianzad vs Macy Chiasson II,Pannie Kianzad,Macy Chiasson,Macy Chiasson,UFC Fight Night: Tuivasa vs. Tybura,Mark Smith,Submission,03.16.2024,UFC Apex,no,...,0,1,1,3:54,"5' 7""","5' 11""","66""","72""",Orthodox,Orthodox


In [30]:
#write new data to top of csv
dfOld = pd.read_csv('databaseUpdated-02.04.2024.csv')

updatedData = pd.concat([dfNew, dfOld], ignore_index=True)

updatedData.head()





,fight,redCorner,blueCorner,winner,event,referee,method_of_victory,date,venue,title_fight,...,redCorner_subs_attempted,blueCorner_subs_attempted,round,time,redCorner_height,blueCorner_height,redCorner_reach,blueCorner_reach,redCorner_stance,blueCorner_stance
0,Tai Tuivasa vs Marcin Tybura,Tai Tuivasa,Marcin Tybura,Marcin Tybura,UFC Fight Night: Tuivasa vs. Tybura,Herb Dean,Submission,03.16.2024,UFC Apex,no,...,0.0,1.0,1,4:08,"6' 2""","6' 3""","75""","78""",Southpaw,Orthodox
1,Bryan Battle vs Ange Loosa,Bryan Battle,Ange Loosa,NaN,UFC Fight Night: Tuivasa vs. Tybura,Mike Beltran,Could Not Continue,03.16.2024,UFC Apex,no,...,0.0,0.0,2,1:00,"6' 1""","5' 10""","77""","74""",Orthodox,Orthodox
2,Kennedy Nzechukwu vs Ovince Saint Preux,Ovince Saint Preux,Kennedy Nzechukwu,Ovince Saint Preux,UFC Fight Night: Tuivasa vs. Tybura,Mark Smith,Decision - Split,03.16.2024,UFC Apex,no,...,0.0,0.0,3,5:00,"6' 3""","6' 5""","80""","83""",Southpaw,Southpaw
3,Christian Rodriguez vs Isaac Dulgarian,Christian Rodriguez,Isaac Dulgarian,Christian Rodriguez,UFC Fight Night: Tuivasa vs. Tybura,Chris Tognoni,Decision - Split,03.16.2024,UFC Apex,no,...,0.0,2.0,3,5:00,"5' 7""","5' 7""","71""","71""",Orthodox,Orthodox
4,Pannie Kianzad vs Macy Chiasson II,Pannie Kianzad,Macy Chiasson,Macy Chiasson,UFC Fight Night: Tuivasa vs. Tybura,Mark Smith,Submission,03.16.2024,UFC Apex,no,...,0.0,1.0,1,3:54,"5' 7""","5' 11""","66""","72""",Orthodox,Orthodox


In [31]:
#df to csv file
updatedData.to_csv('toDelete.csv', index=False)

In [32]:
#parse site using alphabet
all_fighter_links = []
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
for i in range(len(alphabet)):
    url=f'http://ufcstats.com/statistics/fighters?char={alphabet[i]}&page=all'
    headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
    
    #site request
    site = requests.get(url, headers=headers)
    soup = BeautifulSoup(site.content, 'html.parser')

    #scrape fighters from each page
    fighterLinks = []
    fighter_links = soup.find_all('a', class_=re.compile("b-link b-link_style_black"))

    #find hrefs
    for link in fighter_links:
        href = link['href']
        if href:
            fighterLinks.append(href)

    #remove link dups
    fighterLinks = list(dict.fromkeys(fighterLinks))
    all_fighter_links.append(fighterLinks)

all_fighter_links = list(itertools.chain.from_iterable(all_fighter_links))
print(f'Fighters Found: {len(all_fighter_links)}')


    
#scrape individual fighter stats
fighters_statistics = []
for i in all_fighter_links:
    try:
        site = requests.get(i, headers=headers)
        soup = BeautifulSoup(site.content, 'html.parser')

        #initialize stats
        name = None
        nickname = None
        wins = None
        losses = None
        draws = None
        height = None
        weight = None
        reach = None
        stance = None
        dOB = None
        sig_strikes_landed_per_min = None
        sig_striking_accuracy = None
        sig_strike_absorbed_per_min = None
        sig_strike_defense = None
        takedown_average = None
        takedown_accuracy = None
        takedown_defense = None
        sub_average = None


        #scrape + clean nickname
        try:
            nick = soup.find('p', class_=re.compile('b-content__Nickname'))
            nickname = nick.text.strip()
        except:
            pass

        #scrape name
        tempName = soup.find_all('span', class_=re.compile("b-content__title-highlight"))
        #clean name
        name = tempName[0].text.strip()

        #scrape records
        tempRecord = soup.find_all('span', class_=re.compile('b-content__title-record'))
        #clean wins + losses + draws
        record = tempRecord[0].text.strip()
        listRecord = record.split('-')
        for idx, ele in enumerate(listRecord):
            listRecord[idx] = ele.replace('Record: ', '')
        wins = listRecord[0]
        losses = listRecord[1]
        draws = listRecord[2]
        
        #clean soup for rest of stats
        i_tags = soup.find_all('i')
        for itags in i_tags:
            itags.decompose()
        
        #clean stats
        tempRest = soup.find_all('li', class_=re.compile('b-list__box-list-item b-list__box-list-item_type_block'))
    
    
        height = tempRest[0].text.strip()
        weight = tempRest[1].text.strip()
        reach = tempRest[2].text.strip()
        stance = tempRest[3].text.strip()
        dOB = tempRest[4].text.strip()
        sig_strikes_landed_per_min = tempRest[5].text.strip()
        sig_striking_accuracy = tempRest[6].text.strip()
        sig_strike_absorbed_per_min = tempRest[7].text.strip()
        sig_strike_defense = tempRest[8].text.strip()
        takedown_average = tempRest[10].text.strip()
        takedown_accuracy = tempRest[11].text.strip()
        takedown_defense = tempRest[12].text.strip()
        sub_average = tempRest[13].text.strip()
        
        #notification of scrape
        print(f'Scraping {name}...')
        print(i)


        #adding stats to fighters_statistics to prepare for csv
        fighters_statistics.append([name, nickname, wins, losses, draws, height, weight, reach, stance, dOB, sig_strikes_landed_per_min, sig_striking_accuracy, sig_strike_absorbed_per_min, sig_strike_defense, takedown_average, takedown_accuracy, takedown_defense, sub_average])
    except:
        pass
    


    

    #create csv file
    head = ['name', 'nickname', 'wins', 'losses', 'draws', 'height', 'weight', 'reach', 'stance', 'dOB', 'sig_strikes_landed_per_min', 'sig_striking_accuracy_%', 'sig_strike_absorbed_per_min', 'sig_strike_defense(%_of_sig_strikes_not_landed_by_opponent)', 'takedown_average(average_takedown_landed_per_fifteen_min)', 'takedown_accuracy_%', 'takedown_defense(%_of_opponent_takedown_not_landed)', 'sub_average(average_subs_attempted_per_15_mins)']

    with open('ufc_fighters_statistics-03.20.2024.csv', 'w', encoding='UTF8', newline='') as scrapedStats:
        writer = csv.writer(scrapedStats)
        writer.writerow(head)
        writer.writerows(fighters_statistics)

Fighters Found: 4115
Scraping Tom Aaron...
http://ufcstats.com/fighter-details/93fe7332d16c6ad9
Scraping Danny Abbadi...
http://ufcstats.com/fighter-details/15df64c02b6b0fde
Scraping Nariman Abbasov...
http://ufcstats.com/fighter-details/59a9d6dac61c2540
Scraping David Abbott...
http://ufcstats.com/fighter-details/b361180739bed4b0
Scraping Hamdy Abdelwahab...
http://ufcstats.com/fighter-details/3329d692aea4dc28
Scraping Shamil Abdurakhimov...
http://ufcstats.com/fighter-details/2f5cbecbbe18bac4
Scraping Hiroyuki Abe...
http://ufcstats.com/fighter-details/c0ed7b208197e8de
Scraping Daichi Abe...
http://ufcstats.com/fighter-details/5140122c3eecd307
Scraping Papy Abedi...
http://ufcstats.com/fighter-details/c9f6385af6df66d7
Scraping Ricardo Abreu...
http://ufcstats.com/fighter-details/aa6e591c2a2cdecd
Scraping Klidson Abreu...
http://ufcstats.com/fighter-details/7279654c7674cd24
Scraping Cyborg Abreu...
http://ufcstats.com/fighter-details/f689bd7bbd14b392
Scraping Daniel Acacio...
http://u

In [35]:
#site request
url = "https://www.ufc.com/athletes/all"

querystring = {"page":"0"}

payload = ""
headers = {
    "cookie": "STYXKEY_region=USA.US.en.Default",
    "User-Agent": "insomnia/8.5.0"
}

response = requests.request("GET", url, data=payload, headers=headers, params=querystring)


#create soup
soup = BeautifulSoup(response.content, 'html.parser')
div = soup.find('div', class_=re.compile('althelete-total'))

#get num athletes
athleteTotal = div.text.strip().split(' ')[0]
print(f"Athletes found: {athleteTotal}")

#calc number of pages to loop through - 11 fighters shown per
numPages = math.ceil(int(athleteTotal)/11)
print(numPages)

linkParts = []
for i in range(numPages):
    #site request
    url = "https://www.ufc.com/athletes/all"
    querystring = {"page":f"{i}"}

    payload = ""
    headers = {
        "cookie": "STYXKEY_region=USA.US.en.Default",
        "User-Agent": "insomnia/8.5.0"
    }

    response = requests.request("GET", url, data=payload, headers=headers, params=querystring)

    soup = BeautifulSoup(response.content, 'html.parser')
    urlParts = soup.find_all('a', class_=re.compile("e-button--black"))



    for part in urlParts:
        href = part['href']
        if href:
            linkParts.append(href)

print(f"Fighter links found: {len(linkParts)}")
    

fighterStats = []
for part in linkParts:
    try:
        #scrape statistics
        url = f"https://www.ufc.com{part}"
        site = requests.get(url, headers=headers)

        #initalize attributes
        name = None
        nickname = None
        wins = None
        losses = None
        draws = None
        sig_str_accuracy = None
        sig_str_totals = None
        takedown_accuracy = None
        takedown_totals = None
        sig_str_per_minute = None
        takedown_avg_per_fifteen = None
        sig_str_defense = None
        knockdown_avg = None
        sig_str_absorbed_per_min = None
        submission_avg_per_fifteen = None
        takedown_defense = None
        avg_fight_time = None
        nation = None
        age = None
        height = None
        reach = None

        #soup
        soup = BeautifulSoup(site.content, 'html.parser')

        #scrape + clean name
        try:
            name = soup.find('h1', class_=re.compile('hero-profile__name')).text.strip()
        except:
            pass
        

        #scrape + clean nickname
        try:
            nick = soup.find('p', class_=re.compile('hero-profile__nickname'))
            nickname = nick.text.strip()
        except:
            pass
        #scrape + clean wins, losses, draws
        try:
            record = soup.find('p', class_=re.compile('hero-profile__division-body')).text.strip().split(' ')
            record = record[0].split('-')
            wins = record[0]
            losses = record[1]
            draws = record[2]
        except:
            pass

        #scrape + clean sig_str_accuracy + sig_str_totals + takedown_accuracy + takedown_totals
        try:
            div = soup.find_all('div', class_=re.compile("overlap-athlete-content overlap-athlete-content--horizontal"))
            stripped = div[0].text.strip().split('\n')
            clean_stripped = [item for item in stripped if item != '']
        except:
            pass
        try:
            if(clean_stripped[2].lower() == 'striking accuracy'):
                sig_str_accuracy = clean_stripped[1]
        except:
            pass
        try:
            if(clean_stripped[3].lower() == "sig. strikes landed" and clean_stripped[5].lower() == "sig. strikes attempted"):
                sig_str_totals = f"{clean_stripped[4]} of {clean_stripped[6]}"
        except:
            pass
        try:
            stripped = div[1].text.strip().split('\n')
            clean_stripped = [item for item in stripped if item != '']
        except:
            pass
        try:
            if clean_stripped[2].lower() == 'takedown accuracy':
                takedown_accuracy = clean_stripped[1]
        except:
            pass
        try:
            if clean_stripped[3].lower() == 'takedowns landed' and clean_stripped[5].lower() == "takedowns attempted":
                takedown_totals = f"{clean_stripped[4]} of {clean_stripped[6]}"
        except:
            pass



        #scrape + clean sig_str_per_minute + takedown_avg_per_fifteen + sig_str_defense + knockdown_avg

        divs = soup.find_all('div', class_=re.compile('c-stat-compare__group c-stat-compare__group-1'))

        for div in divs:
            try:
                if(div.find('div', class_=re.compile('c-stat-compare__label')).text.strip().lower() == "sig. str. landed"):
                    sig_str_per_minute = div.find('div', class_=re.compile("c-stat-compare__number")).text.strip()
            except:
                pass
            try:
                if(div.find('div', class_=re.compile('c-stat-compare__label')).text.strip().lower() == "takedown avg"):
                    takedown_avg_per_fifteen = div.find('div', class_=re.compile("c-stat-compare__number")).text.strip()
            except:
                pass
            try:
                if(div.find('div', class_=re.compile('c-stat-compare__label')).text.strip().lower() == "sig. str. defense"):
                    sig_str_defense = str(div.find('div', class_=re.compile("c-stat-compare__number")).text.strip().split("\n")[0]) +"%"
            except:
                pass
            try:
                if(div.find('div', class_=re.compile('c-stat-compare__label')).text.strip().lower() == "knockdown avg"):
                    knockdown_avg = div.find('div', class_=re.compile("c-stat-compare__number")).text.strip()
            except:
                pass


        # scrape + clean sig_str_absorbed_per_min + submission_avg_per_fifteen + takedown_defense + avg_fight_time
        divs = soup.find_all('div', re.compile('c-stat-compare__group c-stat-compare__group-2'))
        for div in divs:
            try:
                if(div.find('div', class_=re.compile('c-stat-compare__label')).text.strip().lower() == "sig. str. absorbed"):
                    sig_str_absorbed_per_min = div.find('div', class_=re.compile("c-stat-compare__number")).text.strip()
            except:
                pass
            try:
                if(div.find('div', class_=re.compile('c-stat-compare__label')).text.strip().lower() == "submission avg"):
                    submission_avg_per_fifteen = div.find('div', class_=re.compile("c-stat-compare__number")).text.strip()
            except:
                pass
            try:
                if(div.find('div', class_=re.compile('c-stat-compare__label')).text.strip().lower() == "takedown defense"):
                    takedown_defense = str(div.find('div', class_=re.compile("c-stat-compare__number")).text.strip().split('\n')[0]) +"%"
            except:
                pass
            try:
                if(div.find('div', class_=re.compile('c-stat-compare__label')).text.strip().lower() == "average fight time"):
                    avg_fight_time = div.find('div', class_=re.compile("c-stat-compare__number")).text.strip()
            except:
                pass
                


        #scrape + clean nation + age + height + reach
        divs = soup.find_all('div', class_=re.compile('c-bio__field'))

        for div in divs:
            try:
                if(div.find('div', class_=re.compile('c-bio__label')).text.strip().lower() == 'place of birth'):
                    nation = div.find('div', class_=re.compile('c-bio__text')).text.strip()
            except:
                pass
            try:
                if(div.find('div', class_=re.compile('c-bio__label')).text.strip().lower() == 'age'):
                    age = div.find('div', class_=re.compile('field field--name-age field--type-integer field--label-hidden field__item')).text.strip()
            except:
                pass
            try:
                if(div.find('div', class_=re.compile('c-bio__label')).text.strip().lower() == 'height'):
                    height = div.find('div', class_=re.compile('c-bio__text')).text.strip()
            except:
                pass
            try:
                if(div.find('div', class_=re.compile('c-bio__label')).text.strip().lower() == 'reach'):
                    reach = div.find('div', class_=re.compile('c-bio__text')).text.strip()
            except:
                pass


        print(f'Scraping {name}...')
        print(url)
        fighterStats.append([name, nickname, wins, losses, draws, height, reach, age, nation, sig_str_accuracy, sig_str_totals, takedown_accuracy, takedown_totals, sig_str_per_minute, takedown_avg_per_fifteen, sig_str_defense, knockdown_avg, sig_str_absorbed_per_min, submission_avg_per_fifteen, takedown_defense, avg_fight_time])
    except:
        pass


#create csv file

head = ['name', 'nickname', 'wins', 'losses', 'draws', 'height', 'reach', 'age', 'nation', 'sig_str_accuracy', 'sig_str_totals', 'takedown_accuracy', 'takedown_totals', 'sig_str_per_minute', 'takedown_avg_per_fifteen', 'sig_str_defense', 'knockdown_avg', 'sig_str_absorbed_per_min', 'submission_avg_per_fifteen', 'takedown_defense', 'avg_fight_time']

with open('alt_fighter_stats-03.20.2024.csv', 'w', encoding='UTF8', newline='') as scrapedFighters:
    writer = csv.writer(scrapedFighters)
    writer.writerow(head)
    writer.writerows(fighterStats)

Athletes found: 2947
268
Fighter links found: 2947
Scraping Danny Abbadi...
https://www.ufc.com/athlete/danny-abbadi
Scraping Nariman Abbassov...
https://www.ufc.com/athlete/nariman-abbassov
Scraping Tank Abbott...
https://www.ufc.com/athlete/tank-abbott
Scraping Hamdy Abdelwahab...
https://www.ufc.com/athlete/hamdy-abdelwahab
Scraping Shamil Abdurakhimov...
https://www.ufc.com/athlete/shamil-abdurakhimov
Scraping Daichi Abe...
https://www.ufc.com/athlete/daichi-abe
Scraping Papy Abedi...
https://www.ufc.com/athlete/papy-abedi
Scraping Klidson Abreu...
https://www.ufc.com/athlete/klidson-abreu
Scraping Ricardo Abreu...
https://www.ufc.com/athlete/ricardo-abreu
Scraping John Adajar...
https://www.ufc.com/athlete/john-adajar
Scraping Juan Adams...
https://www.ufc.com/athlete/juan-adams
Scraping Anthony Adams...
https://www.ufc.com/athlete/anthony-adams
Scraping Scott Adams...
https://www.ufc.com/athlete/tenk-ebbott
Scraping Zarrukh Adashev...
https://www.ufc.com/athlete/zarrukh-adashev
S

In [62]:
df = pd.read_csv('ufc_fighters_statistics-03.20.2024.csv')
dfAlt = pd.read_csv('alt_fighter_stats-03.20.2024.csv')

df.head()

,name,nickname,wins,losses,draws,height,weight,reach,stance,dOB,sig_strikes_landed_per_min,sig_striking_accuracy_%,sig_strike_absorbed_per_min,sig_strike_defense(%_of_sig_strikes_not_landed_by_opponent),takedown_average(average_takedown_landed_per_fifteen_min),takedown_accuracy_%,takedown_defense(%_of_opponent_takedown_not_landed),sub_average(average_subs_attempted_per_15_mins)
0,Tom Aaron,NaN,5,3,0,--,155 lbs.,--,NaN,"Jul 13, 1978",0.00,0%,0.00,0%,0.00,0%,0%,0.0
1,Danny Abbadi,The Assassin,4,6,0,"5' 11""",155 lbs.,--,Orthodox,"Jul 03, 1983",3.29,38%,4.41,57%,0.00,0%,77%,0.0
2,Nariman Abbasov,Bayraktar,28,4,0,"5' 8""",155 lbs.,"66""",Orthodox,"Feb 01, 1994",3.00,20%,5.67,46%,0.00,0%,66%,0.0
3,David Abbott,Tank,10,15,0,"6' 0""",265 lbs.,--,Switch,--,1.35,30%,3.55,38%,1.07,33%,66%,0.0
4,Hamdy Abdelwahab,The Hammer,5,0,0 (1 NC),"6' 2""",264 lbs.,"72""",Southpaw,"Jan 22, 1993",3.87,52%,3.13,59%,3.00,75%,0%,0.0


In [63]:
dfAlt.head()

,name,nickname,wins,losses,draws,height,reach,age,nation,sig_str_accuracy,...,takedown_accuracy,takedown_totals,sig_str_per_minute,takedown_avg_per_fifteen,sig_str_defense,knockdown_avg,sig_str_absorbed_per_min,submission_avg_per_fifteen,takedown_defense,avg_fight_time
0,Danny Abbadi,"""The Assassin""",2.0,2.0,0.0,71.0,NaN,39.0,"Orlando, United States",38%,...,NaN,NaN,3.29,0.00,58%,0.00,4.41,0.00,78%,08:58
1,Nariman Abbassov,NaN,0.0,1.0,0.0,NaN,NaN,29.0,Kazakhstan,20%,...,0%,NaN,3.00,0.00,46%,0.00,5.67,0.00,67%,15:00
2,Tank Abbott,"""Tank""",8.0,10.0,0.0,72.0,NaN,NaN,"Huntington Beach, United States",39%,...,NaN,NaN,2.41,0.00,38%,0.00,10.03,0.00,67%,01:40
3,Hamdy Abdelwahab,"""The Hammer""",5.0,0.0,0.0,74.0,72.0,30.0,Egypt,53%,...,75%,3 of 4,3.87,3.00,59%,1.00,3.13,0.00,NaN,15:00
4,Shamil Abdurakhimov,"""Abrek""",20.0,8.0,0.0,75.0,76.0,41.0,"Dagestan Republic, Russia",44%,...,23%,5 of 30,2.41,1.01,55%,0.29,3.02,0.14,45%,09:27


In [64]:
print(len(df))
print(len(dfAlt))

4115
2940


In [65]:
#clean nicknames in dfAlt
dfAlt['nickname'] = dfAlt['nickname'].str.replace('"', '')

In [66]:
#giving nans characters so dups with nans for nickname cannot match
df['nickname'].fillna('!', inplace=True)
dfAlt['nickname'].fillna('~', inplace=True)

/var/folders/pw/f4_6q0l977bcqprlylly2d1w0000gq/T/ipykernel_843/2480345123.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['nickname'].fillna('!', inplace=True)
/var/folders/pw/f4_6q0l977bcqprlylly2d1w0000gq/T/ipykernel_843/2480345123.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves a

In [67]:
df.head()

,name,nickname,wins,losses,draws,height,weight,reach,stance,dOB,sig_strikes_landed_per_min,sig_striking_accuracy_%,sig_strike_absorbed_per_min,sig_strike_defense(%_of_sig_strikes_not_landed_by_opponent),takedown_average(average_takedown_landed_per_fifteen_min),takedown_accuracy_%,takedown_defense(%_of_opponent_takedown_not_landed),sub_average(average_subs_attempted_per_15_mins)
0,Tom Aaron,!,5,3,0,--,155 lbs.,--,NaN,"Jul 13, 1978",0.00,0%,0.00,0%,0.00,0%,0%,0.0
1,Danny Abbadi,The Assassin,4,6,0,"5' 11""",155 lbs.,--,Orthodox,"Jul 03, 1983",3.29,38%,4.41,57%,0.00,0%,77%,0.0
2,Nariman Abbasov,Bayraktar,28,4,0,"5' 8""",155 lbs.,"66""",Orthodox,"Feb 01, 1994",3.00,20%,5.67,46%,0.00,0%,66%,0.0
3,David Abbott,Tank,10,15,0,"6' 0""",265 lbs.,--,Switch,--,1.35,30%,3.55,38%,1.07,33%,66%,0.0
4,Hamdy Abdelwahab,The Hammer,5,0,0 (1 NC),"6' 2""",264 lbs.,"72""",Southpaw,"Jan 22, 1993",3.87,52%,3.13,59%,3.00,75%,0%,0.0


In [70]:
#find special characters in df
names = (df['name'].values)
nicknames = (df['nickname'].values)

chars2rep = []
for name in names:
    if(isinstance(name, str)):
        for char in name:
            if char.lower() not in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ']:
                chars2rep.append(char)
            if char.upper() not in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', "Q", 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ' ']:
                chars2rep.append(char)
for nickname in nicknames:
    if(isinstance(nickname, str)):
        for char in nickname:
            if char.lower() not in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ']:
                chars2rep.append(char)
            if char.upper() not in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', "Q", 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ' ']:
                chars2rep.append(char)


chars2rep = list(set(chars2rep))
for char in chars2rep:
    print(char)

!


In [69]:
#clean name, nickname
for index, row in df.iterrows():
    name = row['name']
    nickname = row['nickname']
    if(isinstance(name, str)):
        cleanName = name.replace('?', '').replace('4', '').replace('9', '').replace('0', '').replace('5', '').replace('7', '').replace('1', '').replace(',', '').replace('3', '').replace('%', '').replace("'", '').replace('-', '').replace('2', '').replace('.', '')
        df.loc[index, 'name'] = cleanName
    if(isinstance(nickname, str)):
        cleanNickname = nickname.replace('?', '').replace('4', '').replace('9', '').replace('0', '').replace('5', '').replace('7', '').replace('1', '').replace(',', '').replace('3', '').replace('%', '').replace("'", '').replace('-', '').replace('2', '').replace('.', '')
        df.loc[index, 'nickname'] = cleanNickname

In [73]:
#find special characters in df
names = (dfAlt['name'].values)
nicknames = (dfAlt['nickname'].values)

chars2rep = []
for name in names:
    if(isinstance(name, str)):
        for char in name:
            if char.lower() not in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ']:
                chars2rep.append(char)
            if char.upper() not in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', "Q", 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ' ']:
                chars2rep.append(char)
for nickname in nicknames:
    if(isinstance(nickname, str)):
        for char in nickname:
            if char.lower() not in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ']:
                chars2rep.append(char)
            if char.upper() not in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', "Q", 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ' ']:
                chars2rep.append(char)


chars2rep = list(set(chars2rep))
for char in chars2rep:
    print(char)

~


In [72]:
#clean name, nickname
for index, row in dfAlt.iterrows():
    name = row['name']
    nickname = row['nickname']
    if(isinstance(name, str)):
        cleanName = name.replace('2', '').replace('(', '').replace('3', '').replace('ú', 'u').replace('ç', 'c').replace('Á', 'A').replace('7', '').replace('í', 'i').replace('0', '').replace('ě', 'e').replace('4', '').replace("'", '').replace('%', '').replace('"', '').replace('á', 'a').replace(',', '').replace('ć', 'c').replace('ñ', 'n').replace(')', '').replace('ł', 'l').replace('-', '').replace('8', '').replace('ê', 'e').replace('1', '').replace('ę', 'e').replace('5', '').replace('š', '').replace('”', '').replace('ř', 'r').replace('î', 'i').replace('ö', '').replace('ã', 'a').replace('ň', 'n').replace('é', 'e').replace('.', '').replace('“', '')
        dfAlt.loc[index, 'name'] = cleanName
    if(isinstance(nickname, str)):
        cleanNickname = nickname.replace('2', '').replace('(', '').replace('3', '').replace('ú', 'u').replace('ç', 'c').replace('Á', 'A').replace('7', '').replace('í', 'i').replace('0', '').replace('ě', 'e').replace('4', '').replace("'", '').replace('%', '').replace('"', '').replace('á', 'a').replace(',', '').replace('ć', 'c').replace('ñ', 'n').replace(')', '').replace('ł', 'l').replace('-', '').replace('8', '').replace('ê', 'e').replace('1', '').replace('ę', 'e').replace('5', '').replace('š', '').replace('”', '').replace('ř', 'r').replace('î', 'i').replace('ö', '').replace('ã', 'a').replace('ň', 'n').replace('é', 'e').replace('.', '').replace('“', '')
        dfAlt.loc[index, 'nickname'] = cleanNickname

for char in chars2rep:
    for index, row in dfAlt.iterrows():
        name = row['name']
        nickname = row['nickname']
        if(char != '~'):
            if(isinstance(name, str)):
                cleanName = name.replace(char, '')
                dfAlt.loc[index, 'name'] = cleanName
            if(isinstance(nickname, str)):
                cleanNickname = nickname.replace(char, '')
                dfAlt.loc[index, 'nickname'] = cleanNickname

In [74]:
#assign nicknames for matching names found within dataframes
count = 1
for index, row in df.iterrows():
    name = row['name']
    nickname = row['nickname']
    dup_check = df.loc[df['name'] == name]
    dup_check2 = dfAlt.loc[dfAlt['name'] == name]
    if(len(dup_check == 1) and len(dup_check2) == 1):
        try:
            altIndex = dfAlt.loc[dfAlt['name'] == name].index[0]
            altNickname = dfAlt.loc[altIndex, 'nickname']
            if(nickname != altNickname):
                if(nickname == '!' and altNickname == '~'):
                    df.loc[index, 'nickname'] = str(count)
                    dfAlt.loc[altIndex, 'nickname'] = str(count)
                    count+=1
                elif(nickname != '!'):
                    dfAlt.loc[altIndex, 'nickname'] = nickname
                elif(altNickname != '~'):
                    df.loc[index, 'nickname'] = altNickname
        except:
            pass

In [75]:
#outer merge the dfs
dfMerged = pd.merge(df, dfAlt, on=['name', 'nickname'], how='outer', indicator=True)
dfMerged.head()

,name,nickname,wins_x,losses_x,draws_x,height_x,weight,reach_x,stance,dOB,...,takedown_totals,sig_str_per_minute,takedown_avg_per_fifteen,sig_str_defense,knockdown_avg,sig_str_absorbed_per_min,submission_avg_per_fifteen,takedown_defense,avg_fight_time,_merge
0,AJ Cunningham,The Savage,11.0,4.0,0,"5' 10""",155 lbs.,"71""",Orthodox,"Sep 07, 1994",...,NaN,7.66,0.00,49%,0.00,11.90,0.00,33%,06:51,both
1,AJ Dobson,181,7.0,2.0,0 (1 NC),"6' 1""",185 lbs.,"76""",Orthodox,"Jan 18, 1992",...,NaN,4.38,1.82,47%,0.30,5.25,0.30,65%,12:20,both
2,AJ Fletcher,The Ghost,10.0,3.0,0,"5' 10""",170 lbs.,"67""",Switch,"Feb 18, 1997",...,4 of 14,3.36,1.54,47%,0.31,4.61,0.93,33%,09:43,both
3,AJ Fonseca,!,5.0,4.0,0,"5' 4""",145 lbs.,--,NaN,--,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,AJ Matthews,!,8.0,7.0,0,"5' 11""",185 lbs.,--,NaN,--,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [76]:
#query dfMerged for intersections
dfMatches = pd.DataFrame(dfMerged.query('_merge == "both"'))
dfMatches.head()

,name,nickname,wins_x,losses_x,draws_x,height_x,weight,reach_x,stance,dOB,...,takedown_totals,sig_str_per_minute,takedown_avg_per_fifteen,sig_str_defense,knockdown_avg,sig_str_absorbed_per_min,submission_avg_per_fifteen,takedown_defense,avg_fight_time,_merge
0,AJ Cunningham,The Savage,11.0,4.0,0,"5' 10""",155 lbs.,"71""",Orthodox,"Sep 07, 1994",...,NaN,7.66,0.00,49%,0.00,11.90,0.00,33%,06:51,both
1,AJ Dobson,181,7.0,2.0,0 (1 NC),"6' 1""",185 lbs.,"76""",Orthodox,"Jan 18, 1992",...,NaN,4.38,1.82,47%,0.30,5.25,0.30,65%,12:20,both
2,AJ Fletcher,The Ghost,10.0,3.0,0,"5' 10""",170 lbs.,"67""",Switch,"Feb 18, 1997",...,4 of 14,3.36,1.54,47%,0.31,4.61,0.93,33%,09:43,both
7,Aalon Cruz,147,8.0,4.0,0,"6' 0""",145 lbs.,"78""",Switch,"Sep 20, 1989",...,NaN,7.58,0.00,59%,0.85,8.88,0.00,NaN,05:54,both
8,Aaron Brink,95,26.0,26.0,0 (2 NC),"6' 3""",205 lbs.,--,Orthodox,"Nov 12, 1974",...,NaN,0.00,0.00,57%,0.00,19.64,0.00,NaN,00:55,both


In [87]:
dfCareer = pd.read_csv('careerStatsData.csv')
dfCareer.head()

,name,nickname,wins,losses,draws,age,nation,knockdown_avg,sig_str_accuracy,takedown_average,takedown_accuracy,subs_attempted_average,height,reach,stance,sig_str_landed_per_min,average_fight_time,sig_str_absorbed_per_min,sig_str_defense,takedown_defense
0,AJ Dobson,180,7.0,2.0,0 (1 NC),18-Jan-1992,"Brooklyn, United States",0.30,47%,1.82,75%,0.3,"6' 1""","76""",Orthodox,4.38,12:20,5.25,47%,64%
1,AJ Fletcher,The Ghost,10.0,3.0,0,18-Feb-1997,"Baton Rouge, United States",0.31,49%,1.54,35%,0.9,"5' 10""","67""",Switch,3.36,9:43,4.61,46%,33%
2,Aalon Cruz,146,8.0,4.0,0,20-Sep-1989,"Parrish, United States",0.85,39%,0.00,0%,0.0,"6' 0""","78""",Switch,7.58,5:54,8.88,58%,0%
3,Aaron Brink,95,26.0,26.0,0 (2 NC),12-Nov-1974,United States,0.00,42%,0.00,0%,0.0,"6' 3""",NaN,Orthodox,3.49,0:55,5.71,57%,0%
4,Aaron Jeffery,345,10.0,3.0,0,14-Nov-1992,"Tillsonburg, Canada",0.00,45%,0.00,0%,0.0,"6' 2""","73""",Orthodox,3.10,9:12,4.46,34%,50%


In [88]:
print(len(dfCareer))

2648


In [84]:
for index, row, in dfMatches.iterrows():
    name = row['name']
    if name not in dfCareer['name'].values:
        if row.notna().all():
            dfCareer.loc[len(dfCareer)] = row

In [85]:
print(len(dfCareer))

2706


In [86]:
print(dfCareer.loc[2648])

name                        Albert Morales
nickname                       The Warrior
wins                                   NaN
losses                                 NaN
draws                                  NaN
age                                   32.0
nation                       United States
knockdown_avg                          0.0
sig_str_accuracy                       36%
takedown_average                       NaN
takedown_accuracy                      36%
subs_attempted_average                 NaN
height                                 NaN
reach                                  NaN
stance                            Orthodox
sig_str_landed_per_min                 NaN
average_fight_time                     NaN
sig_str_absorbed_per_min              3.96
sig_str_defense                        56%
takedown_defense                       71%
Name: 2648, dtype: object
